In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import holidays



In [ ]:
driver_data = pd.read_csv('Crash_Reporting_-_Drivers_Data_20250425.csv', parse_dates=['Crash Date/Time'])
incident_data = pd.read_csv('Crash_Reporting_-_Incidents_Data_20250425.csv', parse_dates=['Crash Date/Time'])
print('Rows in Driver data', len(driver_data))
print('Rows in Incidents data',len(incident_data))

#Rows in Driver data 195,692
#Rows in Incidents data 110,927

<ipython-input-2-35596cf6d797>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  driver_data = pd.read_csv('Crash_Reporting_-_Drivers_Data_20250425.csv', parse_dates=['Crash Date/Time'])
<ipython-input-2-35596cf6d797>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  driver_data = pd.read_csv('Crash_Reporting_-_Drivers_Data_20250425.csv', parse_dates=['Crash Date/Time'])
<ipython-input-2-35596cf6d797>:2: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  incident_data = pd.read_csv('Crash_Reporting_-_Incidents_Data_20250425.csv', parse_dates=['Crash Date/Time'])
<ipython-input-2-35596cf6d797>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-

Rows in Driver data 195692
Rows in Incidents data 110927


In [ ]:
#Make copy of dataframe as Backup
driver_data_copy = driver_data.copy()
incident_data_copy = incident_data.copy()

In [ ]:
#Merging Crash datasets

merge_on = ['Report Number', 'Local Case Number' ,'Agency Name', 'ACRS Report Type' ,'Crash Date/Time',
            'Route Type' ,'Road Name', 'Cross-Street Name' ,'Off-Road Description' ,'Municipality',
            'Related Non-Motorist' ,'Collision Type','Weather','Surface Condition','Light',
            'Traffic Control','Driver Substance Abuse','Non-Motorist Substance Abuse',
            'Latitude','Longitude','Location']

main_merge = driver_data.merge(incident_data, on= merge_on, how='outer')

print('Number of rows in main merge:', len(main_merge))
# Number of rows in main merge 218207



Number of rows in main merge: 221609


In [ ]:
#Time of day
def categorize_time_of_day(dt):
  hour = dt.hour
  if 3 <= hour < 7:
      return "Early Morning"
  elif 8 <= hour < 12:
       return "Morning"
  elif 12 <= hour < 17:
       return "Afternoon"
  elif 17 <= hour < 21:
       return "Evening"
  else:
       return "Late Night"


#Seasons
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Autumn'

main_merge['Season'] = main_merge['Crash Date/Time'].apply(get_season)


print(main_merge.head())


  Report Number Local Case Number                Agency Name  \
0    DD5502000M          15004827  Rockville Police Departme   
1    DD5502000M          15004827  Rockville Police Departme   
2    DD5502000N          15008023  Rockville Police Departme   
3    DD5502000N          15008023  Rockville Police Departme   
4    DD5502000N          15008023  Rockville Police Departme   

        ACRS Report Type     Crash Date/Time        Route Type  \
0  Property Damage Crash 2015-01-30 12:16:00  Maryland (State)   
1  Property Damage Crash 2015-01-30 12:16:00  Maryland (State)   
2  Property Damage Crash 2015-02-18 14:52:00  Maryland (State)   
3  Property Damage Crash 2015-02-18 14:52:00  Maryland (State)   
4  Property Damage Crash 2015-02-18 14:52:00  Maryland (State)   

        Road Name Cross-Street Name Off-Road Description Municipality  ...  \
0  W JEFFERSON ST   S WASHINGTON ST                  NaN    ROCKVILLE  ...   
1  W JEFFERSON ST   S WASHINGTON ST                  NaN    RO

In [ ]:
#Renaming location column
main_merge = main_merge.rename(columns= {'Location':'Geolocation'})

#Date and Time
main_merge['Year'] = main_merge['Crash Date/Time'].dt.year
main_merge['Month'] = main_merge['Crash Date/Time'].dt.month_name()
main_merge['Time'] = main_merge['Crash Date/Time'].dt.time

#Holiday- MD
us_holidays = holidays.US(state='MD')
main_merge['Holiday'] = main_merge['Crash Date/Time'].apply(lambda x: us_holidays.get(x, "Not a Holiday"))

#Day Type
main_merge['Day Type'] = main_merge['Crash Date/Time'].apply(lambda x: 'Weekend' if x.weekday() >= 5 else 'Weekday')

#Day of Week
main_merge['Day of Week'] = main_merge['Crash Date/Time'].dt.day_name()

#Time of Day
main_merge['Time of Day'] = main_merge['Crash Date/Time'].apply(categorize_time_of_day)

#convert Year from int to str
main_merge['Year']= main_merge['Year'].astype(str)

In [ ]:
#main_merge.info()
#main_merge.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221609 entries, 0 to 221608
Data columns (total 62 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   Report Number                  221609 non-null  object        
 1   Local Case Number              221609 non-null  object        
 2   Agency Name                    221609 non-null  object        
 3   ACRS Report Type               221609 non-null  object        
 4   Crash Date/Time                221609 non-null  datetime64[ns]
 5   Route Type                     199406 non-null  object        
 6   Road Name                      196255 non-null  object        
 7   Cross-Street Name              183014 non-null  object        
 8   Off-Road Description           21067 non-null   object        
 9   Municipality                   20822 non-null   object        
 10  Related Non-Motorist           6964 non-null    object        
 11  

,Report Number,Local Case Number,Agency Name,ACRS Report Type,Crash Date/Time,Route Type,Road Name,Cross-Street Name,Off-Road Description,Municipality,...,Road Alignment,Road Condition,Road Division,Year,Month,Time,Holiday,Day Type,Day of Week,Time of Day
0,DD5502000M,15004827,Rockville Police Departme,Property Damage Crash,2015-01-30 12:16:00,Maryland (State),W JEFFERSON ST,S WASHINGTON ST,NaN,ROCKVILLE,...,STRAIGHT,NO DEFECTS,"TWO-WAY, DIVIDED, UNPROTECTED PAINTED MIN 4 FEET",2015,January,12:16:00,Not a Holiday,Weekday,Friday,Afternoon
1,DD5502000M,15004827,Rockville Police Departme,Property Damage Crash,2015-01-30 12:16:00,Maryland (State),W JEFFERSON ST,S WASHINGTON ST,NaN,ROCKVILLE,...,STRAIGHT,NO DEFECTS,"TWO-WAY, DIVIDED, UNPROTECTED PAINTED MIN 4 FEET",2015,January,12:16:00,Not a Holiday,Weekday,Friday,Afternoon
2,DD5502000N,15008023,Rockville Police Departme,Property Damage Crash,2015-02-18 14:52:00,Maryland (State),W JEFFERSON ST,GREAT FALLS RD,NaN,ROCKVILLE,...,NaN,NaN,NaN,2015,February,14:52:00,Not a Holiday,Weekday,Wednesday,Afternoon
3,DD5502000N,15008023,Rockville Police Departme,Property Damage Crash,2015-02-18 14:52:00,Maryland (State),W JEFFERSON ST,GREAT FALLS RD,NaN,ROCKVILLE,...,STRAIGHT,NO DEFECTS,"TWO-WAY, NOT DIVIDED",2015,February,14:52:00,Not a Holiday,Weekday,Wednesday,Afternoon
4,DD5502000N,15008023,Rockville Police Departme,Property Damage Crash,2015-02-18 14:52:00,Maryland (State),W JEFFERSON ST,GREAT FALLS RD,NaN,ROCKVILLE,...,NaN,NaN,NaN,2015,February,14:52:00,Not a Holiday,Weekday,Wednesday,Afternoon


BELOW WE ARE Replacing VALUES FOR Standardization

In [ ]:
#Crash Replacement for Standardization - Agency Name
#replacements
rockville = 'Rockville Police Department'
takoma = 'Takoma Park Police Department'
gaithersburg = ' Gaithersburg Police Department'
mcpark = 'Maryland-National Capital'
montgomery = 'Montgomery County Police'

#Replacing
replacements_Agency = {"Rockville Police Departme": rockville,
                "ROCKVILLE": rockville,
                'TAKOMA': takoma,
                'Takoma Park Police Depart': takoma,
                'Gaithersburg Police Depar':gaithersburg,
                'GAITHERSBURG':gaithersburg,
                'MCPARK':mcpark,
                'MONTGOMERY':montgomery,
               }

main_merge['Agency Name'] = main_merge['Agency Name'].replace(replacements_Agency)

In [ ]:
#Crash Replacement for Standardization - Route Type
#replacements
md_state = 'Maryland (State) Route'
municip_route = 'Municipality Route'
county_route = 'County Route'
gov_route = 'Government Route'


#Replacing
replacements_Route = {'Maryland (State)': md_state,
                'Municipality': municip_route,
                'County': county_route,
                'Government': gov_route,

               }

main_merge['Route Type'] = main_merge['Route Type'].replace(replacements_Route)

In [ ]:
#Crash Replacement for Standardization - related non-motorist
#replacements
pedestrian ='Pedestrian'
cyclist = 'Cyclist'
wheelchair = 'Wheelchair'
scooter = 'Scooter'
machine_ops = 'Machine Operator'
not_in_transport = 'Stationary Vehicle Occupant'

animal = 'Animal-Drawn Vehicle'
unknown = 'Unknown'
other = 'Other'

#Replacing
replacements_non_motorist = {
                'PEDESTRIAN' : pedestrian,
                'OTHER, PEDESTRIAN' : pedestrian,
                'OTHER CONVEYANCE, PEDESTRIAN' : pedestrian,
                'Other Pedestrian (person in a building, skater, personal conveyance, etc.), Pedestrian' : pedestrian,
                'Other Pedestrian (person in a building, skater, personal conveyance, etc.)' : pedestrian,

                'BICYCLIST' : cyclist,
                'Cyclist (non-electric)' :cyclist,
                'OTHER PEDALCYCLIST' :cyclist,
                'OTHER PEDALCYCLIST' :cyclist,
                'BICYCLIST, PEDESTRIAN' :cyclist,
                'BICYCLIST, OTHER' :cyclist,
                'Cyclist (Electric)' :cyclist,

                'Wheelchair (electric)' :wheelchair,
                'Unknown, Wheelchair (electric)' :wheelchair,
                'Wheelchair (non-electric)' :wheelchair,

                'Scooter (electric)' :scooter,
                'Scooter (non-Electric)' :scooter,

                'OTHER' :other,
                'OTHER, OTHER CONVEYANCE' :other,
                'OTHER CONVEYANCE' :other,

                'MACHINE OPERATOR/RIDER' :machine_ops,
                'MACHINE OPERATOR/RIDER, OTHER' :machine_ops,

                'Occupant of Motor Vehicle Not in Transport' :not_in_transport,
                'Occupant of Motor Vehicle Not in Transport Pedestrian' : not_in_transport ,
                'Occupant of Motor Vehicle Not in Transport, Pedestrian' : not_in_transport ,

                'Unknown' : unknown,
                'Unknown Type Of Non-Motorist' : unknown,

                'IN ANIMAL-DRAWN VEH' : animal
                }




main_merge['Related Non-Motorist'] = main_merge['Related Non-Motorist'].replace(replacements_non_motorist)


In [ ]:
#Crash Replacement for Standardization - Collision Type
#replacements
head_on = 'Head-on'
headon_left = 'Head-on, Left Turn'
rear_end = 'Rear End'
same_swipe = 'Same Direction, Side Swipe'
oppsite_swipe = 'Opposite Direction, Side Swipe'
angle_col ='Angle Collision'

other_unknown = 'Other/Unknown'

#Replacing




                  #Head on
replacements_Collision = { 'HEAD ON': head_on,
                  'Front to Front': head_on,

                  #Head on but with angle
                  'ANGLE MEETS LEFT HEAD ON':headon_left,
                  'HEAD ON LEFT TURN':headon_left,


                  #Left turns
                  'SAME DIR BOTH LEFT TURN': 'Same Direction Left Turn, Both',

                  'SAME DIRECTION LEFT TURN': 'Same Direction Left Turn',

                  'OPPOSITE DIR BOTH LEFT TURN': 'Oppsite Direction Left Turn, Both',

                  #Right turn
                  'SAME DIRECTION RIGHT TURN': 'Same Direction Right Turn',



                  #Rear ends
                  'SAME DIR REAR END':rear_end,
                  'Front to Rear':rear_end,
                  #rear ends while turning left
                  'SAME DIR REND RIGHT TURN': 'Same Direction Rear End, Right Turn',
                  #rear ends while turning right
                  'SAME DIR REND LEFT TURN':'Same Direction Rear End, Left Turn',

                  #These can stay
                  #'Rear To Rear':,
                  #'Rear To Side':,

                  #Sideswipe
                  'SAME DIRECTION SIDESWIPE':same_swipe,
                  'Sideswipe, Same Direction':same_swipe,
                  #Sideswipe - oppsite direction.
                  'OPPOSITE DIRECTION SIDESWIPE':oppsite_swipe,
                  'Sideswipe, Opposite Direction':oppsite_swipe,

                  #Angle Collisons
                  'STRAIGHT MOVEMENT ANGLE':angle_col, #Figure out what this means
                  'Angle': angle_col,


                  'ANGLE MEETS RIGHT TURN': 'Right-Turn Angle Impact', # A vehicle making a right turn is hit at an angle by another vehicle.
                  'ANGLE MEETS LEFT TURN': 'Left-Turn Angle Impact',



                  #Single Vehicale
                  'SINGLE VEHICLE': 'Single Vehicle',


                  # Other and Unknowns
                  'UNKNOWN':other_unknown,
                  'Unknown':other_unknown,

                  'OTHER':other_unknown,
                  'Other':other_unknown ,
               }

main_merge['Collision Type'] = main_merge['Collision Type'].replace(replacements_Collision)

In [ ]:
#Crash Replacement for Standardization - Weather
#replacements
foggy = 'Foggy'


#Replacing
replacements_Weather = { 'CLEAR': 'Clear',
                  'CLOUDY': 'Cloudy',
                  'RAINING':'Rain',
                  'SNOW':'Snow',
                  'WINTRY MIX':'Wintry Mix',
                  'BLOWING SAND, SOIL, DIRT': 'Blowing Sand, Soil, Dirt',
                  'BLOWING SNOW':'Blowing Snow',

                  'FOGGY':foggy,
                  'Fog, Smog, Smoke':foggy,

                  'SEVERE WINDS': 'Severe Crosswinds',
                  'SLEET':'Sleet Or Hail',
                  'Freezing Rain Or Freezing Drizzle': 'Freezing Rain',

                  'UNKNOWN': 'Unknown',
                  'OTHER': 'Other',


               }

main_merge['Weather'] = main_merge['Weather'].replace(replacements_Weather)

In [ ]:
#Crash Replacement for Standardization - Surface Condition

#Replacing
replacements_surface = {
    'DRY': 'Dry',
    'WET': 'Wet',
    'SNOW': 'Snow',
    'ICE': 'Ice/Frost',
    'OTHER': 'Other',
    'SLUSH':  'Slush',
    'MUD, DIRT, GRAVEL':  'Mud, Dirt, Gravel',
    'OIL': 'Oil',
    'WATER(STANDING/MOVING)':  'Water (standing, moving)',
    'Water(standing, moving)':'Water (standing, moving)',
    'SAND': 'Sand',
    'UNKNOWN': 'Unknown'

}

main_merge['Surface Condition'] = main_merge['Surface Condition'].replace(replacements_surface)

In [ ]:
#Crash Replacement for Standardization -Light

#Replacing
replacements_Light = {
    'DAYLIGHT': 'Daylight',
    'DUSK': 'Dusk',
    'DARK LIGHTS ON': 'Dark- Lighted',
    'DAWN' : 'Dawn',
    'DARK NO LIGHTS':'Dark- Not Lighted',
    'UNKNOWN': 'Unknown',
    'OTHER': 'Other',
    'DARK -- UNKNOWN LIGHTING': 'Dark- Unknown Lighting',

 }

main_merge['Light'] = main_merge['Light'].replace(replacements_Light)

In [ ]:
#Crash Replacement for Standardization - Traffic Control
#Standard replacement value- keys
traffic_signal_controller = 'Traffic Signal'
yield_sign = 'Yield Sign'
no_control = 'No Controls'
stop_sign = 'Stop Sign'
person_control = 'Person' #Including flagger, law enforcement, crossing guard, etc
warning_sign = ' Warning Sign'
flashing_signal = 'Flashing Traffic Signal'
ped_crossing_sign = 'Crossing Sign- Pedestrian'
ped_crossing_signal = 'Pedestrian- Traiffic Signal'
ped_crossing_stop_sign = 'Pedestrian- Crossing with Stop Sign'
bike_cross = 'Crossing Sign- Bicycle'
lane_use = 'Lane Use Control Signal'
school_zone = 'School Zone Signs'
rail_road = 'Crossing Signal- Flashing Railroad'
curve_ahead = 'Warning Sign- Curve Ahead'
intersection_ahead = 'Warning Sign- Intersection Ahead'
other_signal = 'Other Signal'



#Variants
traffic_signal_controller_variants = {
    'TRAFFIC SIGNAL',
    'Traffic Control Signal',
    'Other, Traffic Control Signal',
    'Other Signal, Traffic Control Signal',
    'Other Warning Sign, Traffic Control Signal',
    'Flashing Traffic Control Signal',
}

yield_sign_variants = {
    'YIELD SIGN',
    'Yield Sign',
    'No Controls, Yield Sign',
    'Traffic Control Signal, Yield Sign',
}

no_control_variants = {
    'NO CONTROLS',
    'No Controls',
    'No Controls, Other',
}

stop_variants = {
    'STOP SIGN',
    'Stop Sign',
    'No Controls, Stop Sign',
    'Stop Sign, Traffic Control Signal',
}

people_variants = {
    'PERSON',
    'Person (including flagger, law enforcement, crossing guard, etc.',
    'No Controls, Person (including flagger, law enforcement, crossing guard, etc.',
    'Person (including flagger, law enforcement, crossing guard, etc., Traffic Control Signal',
}

warning_sign_variants = {
    'WARNING SIGN',
    'Other Warning Sign',
    'Other, Other Warning Sign'
    'No Controls, Other Warning Sign',
}

flashing_signal_variants = {
    'FLASHING TRAFFIC SIGNAL',
    'Flashing Traffic Control Signal',
    'Flashing Traffic Control Signal, Traffic Control Signal',
    'Flashing Traffic Control Signal, Other Signal',
}

pedestrain_sign_variants = {
    'Pedestrian Crossing Sign',
    'Pedestrian Crossing',
    'No Controls, Pedestrian Crossing Sign',
    'Pedestrian Crossing, Pedestrian Crossing Sign'
 }

pedestrian_signal_variants = {
    'Pedestrian Crossing, Traffic Control Signal',
    'Pedestrian Crossing Sign, Traffic Control Signal',
 }

pedestrian_stop_variants = {
    'Pedestrian Crossing, Stop Sign',
    'Pedestrian Crossing Sign, Stop Sign',
 }

bike_variants = {
    'Bicycle Crossing Sign',
    'Bicycle Crossing Sign, No Controls',
 }

lane_use_variants = {
    'Lane Use Control Signal, Other',
    'Lane Use Control Signal, Traffic Control Signal',
    'Lane Use Control Signal',
    'Lane Use Control Signal',
 }
school_zone_variants = {
    'School Zone Sign',
    'SCHOOL ZONE SIGN DEVICE',
    'School Zone',
 }

rail_road_variants = {
    'Flashing Railroad Crossing Signal (may include gates)',
    'Flashing Railroad Crossing Signal (may include gates)',
    'RAILWAY CROSSING DEVICE',
 }

curve_ahead_variants = {
    'Curve Ahead Warning Sign, No Controls',
    'Curve Ahead Warning Sign',
 }

intersection_ahead_variants = {
    'Intersection Ahead Warning Sign, No Controls',
    'Intersection Ahead Warning Sign',
    'Intersection Ahead Warning Sign, Other Warning Sign',
}

other_variants = {
    'Other, Other Signal',
    'No Controls, Other Signal',
    'OTHER'
    'Other'
 }


replacements = {
    **{key: traffic_signal_controller for key in traffic_signal_controller_variants},
    **{key: yield_sign for key in yield_sign_variants},
    **{key: no_control for key in no_control_variants},
    **{key: stop_sign for key in stop_variants},
    **{key: person_control for key in people_variants},
    **{key: warning_sign for key in warning_sign_variants},
    **{key: flashing_signal for key in flashing_signal_variants},
    **{key: ped_crossing_sign for key in pedestrain_sign_variants},
    **{key: ped_crossing_signal for key in pedestrian_signal_variants},
    **{key: ped_crossing_stop_sign for key in pedestrian_stop_variants},
    **{key: bike_cross for key in bike_variants},
    **{key: lane_use for key in lane_use_variants},
    **{key: school_zone for key in school_zone_variants},
    **{key: rail_road for key in rail_road_variants},
    **{key: curve_ahead for key in curve_ahead_variants},
    **{key: intersection_ahead for key in intersection_ahead_variants},

    #**{key: other_signal for key in other_variants},
}


main_merge['Traffic Control'] = main_merge['Traffic Control'].replace(replacements)

In [ ]:
#Crash Replacement for Standardization - Driver Substance Abuse

#Standard replacement value- keys
alcohol_present = 'Alcohol Present'
alcohol_contributed = 'Alcohol Contributed'

combo_contributed = 'Combination Contributed'
combined_sub = 'Combined Substance Present'

medication_contributed = 'Medication Contributed'
medication_present = 'Medication Present'

illegal_contributed = 'Illegal Drug Contributed'
illegal_present = 'Illegal Drug Present'

not_suspected = 'Not Suspect of Alcohol or Drug Use'
none_detected = 'Nothing Detected'

unknown_sub_abu = 'Unknown'
na_other = 'Other'

sus_alcohol_drug = 'Suspect of Alcohol Use, Suspect of Drug Use'
sus_alcohol = 'Suspect of Alcohol Use'
sus_drug = 'Suspect of Drug Use'

non_alcohol_sus_drug = 'Not Suspect of Alcohol Use, Suspect of Drug Use'
sus_alcohol_not_drug = 'Suspect of Alcohol Use, Not Drug Use'

not_sus_alcohol = 'Not Suspect of Alcohol Use'
not_sus_drug = 'Not Suspect of Drug Use'


#Variants
alcohol_present_variants = {
    'ALCOHOL PRESENT',
    'ALCOHOL PRESENT, N/A',
    'ALCOHOL PRESENT, NONE DETECTED',
    'ALCOHOL PRESENT, NONE DETECTED, UNKNOWN',
    'ALCOHOL PRESENT, UNKNOWN',
    'ALCOHOL PRESENT, N/A, UNKNOWN',
    'ALCOHOL PRESENT, N/A, NONE DETECTED',
}

alcohol_contributed_variants = {
    'ALCOHOL CONTRIBUTED, NONE DETECTED',
    'ALCOHOL CONTRIBUTED, N/A, NONE DETECTED',
    'ALCOHOL CONTRIBUTED',
    'ALCOHOL CONTRIBUTED, N/A',
    'ALCOHOL CONTRIBUTED, UNKNOWN',
    'ALCOHOL CONTRIBUTED, ALCOHOL PRESENT',
    'ALCOHOL CONTRIBUTED, NONE DETECTED, UNKNOWN',
}

combo_contributed_variants = {
    'COMBINATION CONTRIBUTED',
    'COMBINATION CONTRIBUTED, NONE DETECTED',
    'COMBINATION CONTRIBUTED, N/A',
}

combined_sub_variants = {
    'COMBINED SUBSTANCE PRESENT, N/A',
    'COMBINED SUBSTANCE PRESENT',
    'COMBINED SUBSTANCE PRESENT, NONE DETECTED',
    'COMBINED SUBSTANCE PRESENT, UNKNOWN',
    'COMBINED SUBSTANCE PRESENT, N/A, NONE DETECTED',

}

medication_contributed_variants = {
    'MEDICATION CONTRIBUTED',
    'MEDICATION CONTRIBUTED, UNKNOWN',
    'MEDICATION CONTRIBUTED, NONE DETECTED',
    'MEDICATION CONTRIBUTED, N/A',
    'MEDICATION CONTRIBUTED, NONE DETECTED, UNKNOWN',

}

medication_present_variants = {
    'MEDICATION PRESENT',
    'MEDICATION PRESENT, N/A',
    'MEDICATION PRESENT, NONE DETECTED',
    'MEDICATION PRESENT, NONE DETECTED, UNKNOWN',
    'MEDICATION PRESENT, N/A, UNKNOWN',

}

illegal_contributed_variants = {
    'ILLEGAL DRUG CONTRIBUTED',
    'ILLEGAL DRUG CONTRIBUTED, N/A',
    'ILLEGAL DRUG CONTRIBUTED, NONE DETECTED',

}
illegal_present_variants = {
    'ILLEGAL DRUG PRESENT',
    'ILLEGAL DRUG PRESENT, N/A',
    'ILLEGAL DRUG PRESENT, NONE DETECTED',
    'ILLEGAL DRUG PRESENT, N/A, NONE DETECTED',
    'ILLEGAL DRUG PRESENT, UNKNOWN',
}

not_suspected_variants = {
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown, Unknown, Unknown, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown, Unknown, Unknown, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Unknown',
    'Not Suspect of Alcohol Use, Unknown, Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Suspect of Drug Use',
    'Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',



}

none_detected_variants = {
    'NONE DETECTED',
    'N/A, NONE DETECTED',
    'N/A, NONE DETECTED, UNKNOWN',
    'NONE DETECTED, OTHER',
    'UNKNOWN', 'NONE DETECTED, UNKNOWN',
}

unknown_sub_abu_vaiants = {
    'N/A, UNKNOWN',
    'Unknown, Unknown',
    'Unknown, Unknown, Unknown, Unknown',
    'Unknown, Unknown, Unknown, Unknown, Unknown, Unknown',
    'Unknown, Unknown, Unknown, Unknown, Unknown, Unknown, Unknown, Unknown',


}

na_other_variants = {
    'N/A, OTHER',
    'OTHER',
}


replacements_both_present = {
    'ALCOHOL PRESENT, ILLEGAL DRUG PRESENT': 'Both Alcohol and Drug Present',}

sus_alcohol_drug_variants = {
    'Suspect of Alcohol Use, Suspect of Drug Use, Unknown, Not Suspect of Drug Use',
    'Suspect of Alcohol Use, Suspect of Drug Use, Unknown, Unknown',
}

sus_drug_variants ={
     'Unknown, Suspect of Drug Use',
}
sus_alcohol_variants = {
    'Suspect of Alcohol Use, Unknown',

}

non_alcohol_sus_drug_variants = {
    'Not Suspect of Alcohol Use, Suspect of Drug Use, Unknown, Unknown',
    'Not Suspect of Alcohol Use, Suspect of Drug Use, Not Suspect of Alcohol Use, Not Suspect of Drug Use',

}

sus_alcohol_not_drug_variants= {
    'Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Unknown',
    'Suspect of Alcohol Use, Not Suspect of Drug Use, Suspect of Alcohol Use, Not Suspect of Drug Use',
    'Suspect of Alcohol Use, Not Suspect of Drug Use, Unknown, Not Suspect of Drug Use',
}

not_sus_alcohol_variants = {
    'Not Suspect of Alcohol Use, Unknown',
    'Not Suspect of Alcohol Use, Unknown, Not Suspect of Alcohol Use, Unknown, Not Suspect of Alcohol Use, Unknown',


}

not_sus_drug_variants = {
    'Unknown, Unknown, Unknown, Not Suspect of Drug Use',
    'Unknown, Not Suspect of Drug Use, Unknown, Unknown',
    'Unknown, Not Suspect of Drug Use'
}


#Replacing
replacements_driver_sub_abuse_L = {
    **{key: alcohol_contributed for key in alcohol_contributed_variants},
    **{key: alcohol_present for key in alcohol_present_variants},
    **{key: combo_contributed for key in combo_contributed_variants},
    **{key: combined_sub for key in combined_sub_variants},
    **{key: medication_contributed for key in medication_contributed_variants},
    **{key: medication_present for key in medication_present_variants},
    **{key: illegal_contributed for key in illegal_contributed_variants},
    **{key: illegal_present for key in illegal_present_variants},
    **{key: not_suspected for key in not_suspected_variants},
    **{key: none_detected for key in none_detected_variants},
    **{key: unknown_sub_abu for key in unknown_sub_abu_vaiants},
    **{key: na_other for key in na_other_variants},

    **{key: sus_alcohol_drug for key in sus_alcohol_drug_variants},
    **{key: sus_alcohol for key in sus_alcohol_variants},
    **{key: sus_drug for key in sus_drug_variants},
    **{key: non_alcohol_sus_drug for key in non_alcohol_sus_drug_variants},
    **{key: sus_alcohol_not_drug for key in sus_alcohol_not_drug_variants},

    **{key: not_sus_alcohol for key in not_sus_alcohol_variants},
    **{key: not_sus_drug for key in not_sus_drug_variants},




 }
main_merge['Driver Substance Abuse'] = main_merge['Driver Substance Abuse'].replace(replacements_both_present)
main_merge['Driver Substance Abuse'] = main_merge['Driver Substance Abuse'].replace(replacements_driver_sub_abuse_L)

In [ ]:
#Crash Replacement for Standardization - Non-Motorist Substance Abuse
#Replacing
replacements_non_mot_sub_abu = {
    **{key: alcohol_contributed for key in alcohol_contributed_variants},
    **{key: alcohol_present for key in alcohol_present_variants},
    **{key: combo_contributed for key in combo_contributed_variants},
    **{key: combined_sub for key in combined_sub_variants},
    **{key: medication_contributed for key in medication_contributed_variants},
    **{key: medication_present for key in medication_present_variants},
    **{key: illegal_contributed for key in illegal_contributed_variants},
    **{key: illegal_present for key in illegal_present_variants},
    **{key: not_suspected for key in not_suspected_variants},
    **{key: none_detected for key in none_detected_variants},
    **{key: unknown_sub_abu for key in unknown_sub_abu_vaiants},
    **{key: na_other for key in na_other_variants},

    **{key: sus_alcohol_drug for key in sus_alcohol_drug_variants},
    **{key: sus_alcohol for key in sus_alcohol_variants},
    **{key: sus_drug for key in sus_drug_variants},
    **{key: non_alcohol_sus_drug for key in non_alcohol_sus_drug_variants},
    **{key: sus_alcohol_not_drug for key in sus_alcohol_not_drug_variants},

    **{key: not_sus_alcohol for key in not_sus_alcohol_variants},
    **{key: not_sus_drug for key in not_sus_drug_variants},
}

main_merge['Non-Motorist Substance Abuse'] = main_merge['Non-Motorist Substance Abuse'].replace(replacements_non_mot_sub_abu)

In [ ]:
#Crash Replacement for Standardization - Injury Severity

#Replacing
replacements_injury_sever = {
    'NO APPARENT INJURY': 'No Apparent Injury',
    'POSSIBLE INJURY': 'Possible Injury',
    'SUSPECTED MINOR INJURY':'Suspected Minor Injury',
    'SUSPECTED SERIOUS INJURY': 'Suspected Serious Injury',
    'FATAL INJURY': 'Fatal Injury',

 }

main_merge['Injury Severity'] = main_merge['Injury Severity'].replace(replacements_injury_sever)

In [ ]:
#Crash Replacement for Standardization -Circumstance

replacements_circum_individual = {
  'N/A, WET' : 'Wet Conditions',
  'N/A, TOLL BOOTH/PLAZA RELATED, WET': 'Toll Booth/Plaza Related- Wet Conditions',
  'N/A, TOLL BOOTH/PLAZA RELATED':'Toll Booth/Plaza Related',
  'N/A, V WIPERS|W OTHER ENVIRONMENTAL':'Other Environmental Factors',
  'N/A, NON-HIGHWAY WORK': 'Non-Highway Work',
  'Other Improper Action, Over-Correcting/Over-Steering': 'Over-Correcting/Over-Steering',
  'Other Improper Action, Over-Correcting/Over-Steering, Too Fast For Conditions': 'Over-Correcting/Over-Steering, Too Fast For Conditions',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action': 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Too Fast For Conditions': 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Ran Red Light, Too Fast For Conditions': 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Red Light',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc': 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Swerved or Avoided Due to Wind',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Ran Stop Sign': 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner,  Ran Stop Sign',
  'Disregard Officer Directions, Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions': 'Disregard Officer Directions, Failed to Keep in Proper Lane',
  'Disregard Officer Directions, Failed to Yield Right-of-Way, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions':'Disregard Officer Directions, Failed to Yield Right-of-Way',
  'Failed to Yield Right-of-Way, Other Improper Action, Too Fast For Conditions':'Failed to Yield Right-of-Way, Too Fast For Conditions',
  'Failed to Yield Right-of-Way, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Wrong Way':'Failed to Yield Right-of-Way, Swerved or Avoided Due to Wind',
  'Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions':  'Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action':  'Operated Motor Vehicle in Reckless or Aggressive Manner',
  'Improper Turn, Other Improper Action':'Improper Turn',
  'Improper Backing, Other Improper Action':  'Improper Backing',
  'Improper Backing, Other Improper Action, Over-Correcting/Over-Steering':  'Improper Backing, Over-Correcting/Over-Steering',
  'Improper Passing, Other Improper Action':  'Improper Passing',
  'Improper Passing, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc':'Improper Passing, Swerved or Avoided Due to Wind',
  'Failed To Stop For School Bus, Improper Backing': 'Failed To Stop For School Bus',
  'Other Improper Action, Wrong Way': 'Wrong Way',
  'Other Improper Action, Wrong Side': 'Wrong Side',
  'Other Improper Action, Ran Red Light':'Ran Red Light',
  'Other Improper Action, Ran Stop Sign':'Ran Stop Sign',


# 'Ran Red Light',
# 'Ran Red Light, Wrong Side',
#'Failed to Keep in Proper Lane, Ran Red Light',
#'Failed to Keep in Proper Lane, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Wrong Side',
#'Disregarded Other Traffic Sign, Operated Motor Vehicle in Reckless or Aggressive Manner',
#'Disregarded Other Traffic Sign, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
#'Disregarded Other Traffic Sign, Ran Stop Sign, Too Fast For Conditions',



    }

#
animal = 'Involved an Animal'
animal_wet = 'Involved an Animal- Wet Conditions'

animal_variants = {
  'ANIMAL, V EXHAUST SYSTEM|R OTHER ROAD',
	'ANIMAL, PHYSICAL OBSTRUCTION(S)',
	'ANIMAL, DEBRIS OR OBSTRUCTION',
	'ANIMAL, BACKUP DUE TO NON-RECURRING INCIDENT',
	'ANIMAL, N/A'
}
animal_wet_variants ={
  'ANIMAL, SMOG, SMOKE, WET',
	'ANIMAL, WET',
	'ANIMAL, ICY OR SNOW-COVERED, RAIN, SNOW, RUTS, HOLES, BUMPS, WET',
	'ANIMAL, ICY OR SNOW-COVERED',
	'ANIMAL, RAIN, SNOW, WET',
	'ANIMAL, SLEET, HAIL, FREEZ. RAIN, WET',
	}

#
blowing_sand = 'Blowing Sand, Soil, Dirt'
blowing_sand_wet= 'Blowing Sand, Soil, Dirt- Wet Conditions'

blowing_sand_variants= {
	'BLOWING SAND, SOIL, DIRT, V EXHAUST SYSTEM|R OTHER ROAD, V WIPERS|W OTHER ENVIRONMENTAL',
	'BLOWING SAND, SOIL, DIRT, N/A',
	}
blowing_sand_wet_variants = {
 'BLOWING SAND, SOIL, DIRT, ICY OR SNOW-COVERED, RAIN, SNOW, SEVERE CROSSWINDS',
 'BLOWING SAND, SOIL, DIRT, ICY OR SNOW-COVERED',
 'BLOWING SAND, SOIL, DIRT, RAIN, SNOW, WET',
 'BLOWING SAND, SOIL, DIRT, RAIN, SNOW, SEVERE CROSSWINDS, SLEET, HAIL, FREEZ. RAIN, WET',
	}

#
icy_snow_covered = 'Icy or Snow-Covered'

icy_snow_covered_variants = {
  'ICY OR SNOW-COVERED, RAIN, SNOW',
  'ICY OR SNOW-COVERED, N/A',
  'ICY OR SNOW-COVERED, SLEET, HAIL, FREEZ. RAIN',
  'ICY OR SNOW-COVERED, RAIN, SNOW, V WIPERS|W OTHER ENVIRONMENTAL',
  'ICY OR SNOW-COVERED, SLEET, HAIL, FREEZ. RAIN, V EXHAUST SYSTEM|R OTHER ROAD, WET',
  'ICY OR SNOW-COVERED, RAIN, SNOW, SMOG, SMOKE',
  'ICY OR SNOW-COVERED, SEVERE CROSSWINDS',
  'ICY OR SNOW-COVERED, RAIN, SNOW, SLEET, HAIL, FREEZ. RAIN, WET',
  'ICY OR SNOW-COVERED, SMOG, SMOKE',
  'ICY OR SNOW-COVERED, RAIN, SNOW, SLEET, HAIL, FREEZ. RAIN',
  'ICY OR SNOW-COVERED, N/A, RUTS, HOLES, BUMPS',
  'ICY OR SNOW-COVERED, V WIPERS|W OTHER ENVIRONMENTAL',
  'ICY OR SNOW-COVERED, N/A, WET',
  'ICY OR SNOW-COVERED, RAIN, SNOW, SEVERE CROSSWINDS, SLEET, HAIL, FREEZ. RAIN',
  'ICY OR SNOW-COVERED, RAIN, SNOW, SEVERE CROSSWINDS',
  'ICY OR SNOW-COVERED, RAIN, SNOW, V EXHAUST SYSTEM|R OTHER ROAD',
  'ICY OR SNOW-COVERED, SLEET, HAIL, FREEZ. RAIN, WET',
  'ICY OR SNOW-COVERED, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'ICY OR SNOW-COVERED, RAIN, SNOW, WET',
  'ICY OR SNOW-COVERED, RAIN, SNOW, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'ICY OR SNOW-COVERED, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'ICY OR SNOW-COVERED, SEVERE CROSSWINDS, V WIPERS|W OTHER ENVIRONMENTAL',
  'ICY OR SNOW-COVERED, RAIN, SNOW, SEVERE CROSSWINDS, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  }

#
sleet_hail = 'Sleet, Hail, Wintry Mix'
sleet_hail_variants = {
  'N/A, SLEET, HAIL, FREEZ. RAIN',
  'SLEET, HAIL, FREEZ. RAIN, WET',
  'SLEET, HAIL, FREEZ. RAIN, V EXHAUST SYSTEM|R OTHER ROAD, WET',
  'SLEET, HAIL, FREEZ. RAIN',
  'RAIN, SNOW, SLEET, HAIL, FREEZ. RAIN, WET',
  'RAIN, SNOW, SLEET, HAIL, FREEZ. RAIN, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'SLEET, HAIL, FREEZ. RAIN, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'SLEET, HAIL, FREEZ. RAIN, TRAFFIC CONTROL DEVICE INOPERATIVE, WET',
  'NON-HIGHWAY WORK, SLEET, HAIL, FREEZ. RAIN, WET',
  'N/A, RAIN, SNOW, SLEET, HAIL, FREEZ. RAIN',
  'SLEET, HAIL, FREEZ. RAIN, SMOG, SMOKE, WET',
  'SLEET, HAIL, FREEZ. RAIN, VISION OBSTRUCTION (INCL. BLINDED BY SUN), WET',
  'SLEET, HAIL, FREEZ. RAIN, WET, WORN, TRAVEL-POLISHED SURFACE',
  }

#
debris_obstruction = 'Debris or Obstruction'
debris_obstruction_wet ='Debris or Obstruction- Wet Conditions'

debris_obstruction_variants ={
  'DEBRIS OR OBSTRUCTION, N/A, RUTS, HOLES, BUMPS, SHOULDERS LOW, SOFT, HIGH',
  'DEBRIS OR OBSTRUCTION, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'DEBRIS OR OBSTRUCTION, N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE',
  'DEBRIS OR OBSTRUCTION, N/A, RUTS, HOLES, BUMPS',
  'DEBRIS OR OBSTRUCTION, SEVERE CROSSWINDS',
  'DEBRIS OR OBSTRUCTION, N/A, PHYSICAL OBSTRUCTION(S), RUTS, HOLES, BUMPS',
  'DEBRIS OR OBSTRUCTION, SEVERE CROSSWINDS, V EXHAUST SYSTEM|R OTHER ROAD',
  'DEBRIS OR OBSTRUCTION, N/A',
  'DEBRIS OR OBSTRUCTION, V WIPERS|W OTHER ENVIRONMENTAL',
  'DEBRIS OR OBSTRUCTION, N/A, PHYSICAL OBSTRUCTION(S)',
  'DEBRIS OR OBSTRUCTION, N/A, PHYSICAL OBSTRUCTION(S), ROAD UNDER CONSTRUCTION/MAINTENANCE, RUTS, HOLES, BUMPS',
  }
debris_obstruction_wet_variants ={
  'DEBRIS OR OBSTRUCTION, V EXHAUST SYSTEM|R OTHER ROAD, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'DEBRIS OR OBSTRUCTION, ICY OR SNOW-COVERED, SLEET, HAIL, FREEZ. RAIN',
  'DEBRIS OR OBSTRUCTION, ICY OR SNOW-COVERED, RAIN, SNOW',
  'DEBRIS OR OBSTRUCTION, ICY OR SNOW-COVERED, V WIPERS|W OTHER ENVIRONMENTAL',
  'DEBRIS OR OBSTRUCTION, SLEET, HAIL, FREEZ. RAIN, TRAFFIC CONTROL DEVICE INOPERATIVE, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'DEBRIS OR OBSTRUCTION, SLEET, HAIL, FREEZ. RAIN, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'DEBRIS OR OBSTRUCTION, RAIN, SNOW, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'DEBRIS OR OBSTRUCTION, SEVERE CROSSWINDS, WET',
  'DEBRIS OR OBSTRUCTION, RAIN, SNOW, SEVERE CROSSWINDS, WET',
  'DEBRIS OR OBSTRUCTION, RAIN, SNOW, WET, WORN, TRAVEL-POLISHED SURFACE',
  'DEBRIS OR OBSTRUCTION, RAIN, SNOW, SMOG, SMOKE, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'DEBRIS OR OBSTRUCTION, RAIN, SNOW, WET',
  'DEBRIS OR OBSTRUCTION, RAIN, SNOW',
  'DEBRIS OR OBSTRUCTION, ICY OR SNOW-COVERED, PHYSICAL OBSTRUCTION(S), RAIN, SNOW',
  'DEBRIS OR OBSTRUCTION, N/A, WET',
  'DEBRIS OR OBSTRUCTION, ICY OR SNOW-COVERED, N/A',
  'DEBRIS OR OBSTRUCTION, RAIN, SNOW, SLEET, HAIL, FREEZ. RAIN, VISION OBSTRUCTION (INCL. BLINDED BY SUN), WET',
  'DEBRIS OR OBSTRUCTION, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  }
#
ruts= 'Poor Road Conditions- Ruts/ Holes/ Bumps'
ruts_wet= 'Poor Road Conditions- Ruts/ Holes/ Bumps- Wet Conditions'

ruts_variants={
  'N/A, RUTS, HOLES, BUMPS',
  'N/A, RUTS, HOLES, BUMPS, WORN, TRAVEL-POLISHED SURFACE',
  'N/A, RUTS, HOLES, BUMPS, SHOULDERS LOW, SOFT, HIGH',
  'N/A, RUTS, HOLES, BUMPS, V EXHAUST SYSTEM|R OTHER ROAD',
  'RUTS, HOLES, BUMPS, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  }
ruts_wet_variants={
  'N/A, RUTS, HOLES, BUMPS, WET',
  'RAIN, SNOW, RUTS, HOLES, BUMPS, WET',
  'RAIN, SNOW, RUTS, HOLES, BUMPS',
  }
#
backup_nonrecurring_incident = 'Backup Due to Non-Recurring Incident'
backup_nonrecurring_incident_wet ='Backup Due to Non-Recurring Incident- Wet Conditions'

backup_nonrecurring_incident_variants ={
  'BACKUP DUE TO NON-RECURRING INCIDENT, N/A',
  'BACKUP DUE TO NON-RECURRING INCIDENT, N/A, NON-HIGHWAY WORK',
  'BACKUP DUE TO NON-RECURRING INCIDENT, ROAD UNDER CONSTRUCTION/MAINTENANCE, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'BACKUP DUE TO NON-RECURRING INCIDENT, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'BACKUP DUE TO NON-RECURRING INCIDENT, BACKUP DUE TO PRIOR CRASH, N/A',
  'BACKUP DUE TO NON-RECURRING INCIDENT, N/A, PHYSICAL OBSTRUCTION(S)',
  'BACKUP DUE TO NON-RECURRING INCIDENT, N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE',
  'BACKUP DUE TO NON-RECURRING INCIDENT, BACKUP DUE TO REGULAR CONGESTION, N/A',
  }
backup_nonrecurring_incident_wet_variants = {
  'BACKUP DUE TO NON-RECURRING INCIDENT, N/A, WET',
  'BACKUP DUE TO NON-RECURRING INCIDENT, RAIN, SNOW, WET',
  }
  #**{key: backup_nonrecurring_incident_wet for key in backup_nonrecurring_incident_wet_variants},


#
backup_regular_congestion = 'Backup Due to Regular Congestion'
backup_regular_congestion_wet = 'Backup Due to Regular Congestion- Wet Conditions'

backup_regular_congestion_variants = {
  'BACKUP DUE TO REGULAR CONGESTION, N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE, RUTS, HOLES, BUMPS',
  'BACKUP DUE TO REGULAR CONGESTION, N/A, V EXHAUST SYSTEM|R OTHER ROAD',
  'BACKUP DUE TO REGULAR CONGESTION, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'BACKUP DUE TO REGULAR CONGESTION, N/A',
  'BACKUP DUE TO REGULAR CONGESTION, N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE',
  'BACKUP DUE TO REGULAR CONGESTION, N/A, NON-HIGHWAY WORK',
  'BACKUP DUE TO REGULAR CONGESTION, N/A, RUTS, HOLES, BUMPS',
                                       }
backup_regular_congestion_wet_variants = {
  'BACKUP DUE TO REGULAR CONGESTION, RAIN, SNOW, WET',
  'BACKUP DUE TO REGULAR CONGESTION, RAIN, SNOW',
  'BACKUP DUE TO REGULAR CONGESTION, ICY OR SNOW-COVERED, V WIPERS|W OTHER ENVIRONMENTAL',
  'BACKUP DUE TO REGULAR CONGESTION, ICY OR SNOW-COVERED, N/A',
  'BACKUP DUE TO REGULAR CONGESTION, N/A, WET',
  'BACKUP DUE TO REGULAR CONGESTION, SLEET, HAIL, FREEZ. RAIN, WET',
  'BACKUP DUE TO REGULAR CONGESTION, ICY OR SNOW-COVERED, RAIN, SNOW',
  'BACKUP DUE TO REGULAR CONGESTION, RAIN, SNOW, V EXHAUST SYSTEM|R OTHER ROAD, WET',
  }


#

backup_prior_crash= 'Backup due to Prior Crash'
backup_prior_crash_wet = 'Backup due to Prior Crash- Wet Conditions'

backup_prior_crash_variants ={
  'BACKUP DUE TO PRIOR CRASH, BACKUP DUE TO REGULAR CONGESTION, N/A',
  'BACKUP DUE TO PRIOR CRASH, N/A, PHYSICAL OBSTRUCTION(S)',
  'BACKUP DUE TO PRIOR CRASH, DEBRIS OR OBSTRUCTION, N/A',
  'BACKUP DUE TO PRIOR CRASH, N/A',
  }
backup_prior_crash_wet_variants= {
  'BACKUP DUE TO PRIOR CRASH, N/A, WET',
  'BACKUP DUE TO PRIOR CRASH, ICY OR SNOW-COVERED, N/A',
  'BACKUP DUE TO PRIOR CRASH, RAIN, SNOW, WET',
  }

#

wind = 'Severe Crosswinds'
wind_wet = 'Severe Crosswinds- Wet Conditions'
wind_variants = {
  'SEVERE CROSSWINDS, TRAFFIC CONTROL DEVICE INOPERATIVE',
  'SEVERE CROSSWINDS, V EXHAUST SYSTEM|R OTHER ROAD',
  'N/A, SEVERE CROSSWINDS',
  }
wind_wet_variants ={
  'SEVERE CROSSWINDS, WET',
  'RAIN, SNOW, SEVERE CROSSWINDS, WET',
                     }

#
polished = 'Travel-Polished Surface'
polished_wet='Travel-Polished Surface- Wet Conditions'
polished_variants ={
  'V WIPERS|W OTHER ENVIRONMENTAL, WET, WORN, TRAVEL-POLISHED SURFACE',
  'N/A, V EXHAUST SYSTEM|R OTHER ROAD, WORN, TRAVEL-POLISHED SURFACE',
  'N/A, WORN, TRAVEL-POLISHED SURFACE',
  }
polished_wet_variants={
  'RAIN, SNOW, WORN, TRAVEL-POLISHED SURFACE',
  'N/A, WET, WORN, TRAVEL-POLISHED SURFACE',
  'RAIN, SNOW, WET, WORN, TRAVEL-POLISHED SURFACE',
  }

#
shoulder = 'Shoulder Low/Soft/High'
shoulder_wet='Shoulder Low/Soft/High- Wet Conditions'

shoulder_variants={
  'N/A, SHOULDERS LOW, SOFT, HIGH',
  'RAIN, SNOW, SHOULDERS LOW, SOFT, HIGH',
  'N/A, SHOULDERS LOW, SOFT, HIGH, V EXHAUST SYSTEM|R OTHER ROAD',
  'SHOULDERS LOW, SOFT, HIGH, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  }
shoulder_wet_variants ={
  'SHOULDERS LOW, SOFT, HIGH, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'N/A, SHOULDERS LOW, SOFT, HIGH, WET',
  'RAIN, SNOW, SHOULDERS LOW, SOFT, HIGH, VISION OBSTRUCTION (INCL. BLINDED BY SUN), WET',
                          }

#
smog ='Smog'
smog_variants={ 'N/A, RAIN, SNOW, SMOG, SMOKE',
 'RAIN, SNOW, SMOG, SMOKE, WET',
 'SMOG, SMOKE, WET',
 'N/A, SMOG, SMOKE',
}

#
vision_obstruction = 'Vision Obstruction'
vision_obstruction_wet ='Vision Obstruction- Wet Conditions'

vision_obstruction_variants={
    'N/A, V WIPERS|W OTHER ENVIRONMENTAL, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
    }
vision_obstruction_wet_variants ={
  'N/A, RAIN, SNOW, VISION OBSTRUCTION (INCL. BLINDED BY SUN)', 'N/A, VISION OBSTRUCTION (INCL. BLINDED BY SUN)', 'VISION OBSTRUCTION (INCL. BLINDED BY SUN), WET',
  'RAIN, SNOW, VISION OBSTRUCTION (INCL. BLINDED BY SUN), WET',
  'NON-HIGHWAY WORK, RAIN, SNOW, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'RAIN, SNOW, V EXHAUST SYSTEM|R OTHER ROAD, VISION OBSTRUCTION (INCL. BLINDED BY SUN), WET',
  }

#
physical_obstruction = 'Physical Obstruction'
physical_obstruction_wet = 'Physical Obstruction- Wet Conditions'

physical_obstruction_variants ={
  'N/A, PHYSICAL OBSTRUCTION(S)',
  'PHYSICAL OBSTRUCTION(S), VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'N/A, PHYSICAL OBSTRUCTION(S), RUTS, HOLES, BUMPS',
  'PHYSICAL OBSTRUCTION(S), V WIPERS|W OTHER ENVIRONMENTAL',
  'N/A, PHYSICAL OBSTRUCTION(S), ROAD UNDER CONSTRUCTION/MAINTENANCE, RUTS, HOLES, BUMPS',
  'N/A, PHYSICAL OBSTRUCTION(S), ROAD UNDER CONSTRUCTION/MAINTENANCE',
                                  }
physical_obstruction_wet_variants ={
  'PHYSICAL OBSTRUCTION(S), RAIN, SNOW, WET',
  'N/A, PHYSICAL OBSTRUCTION(S), WET',
  'PHYSICAL OBSTRUCTION(S), RAIN, SNOW',
  'PHYSICAL OBSTRUCTION(S), V EXHAUST SYSTEM|R OTHER ROAD, VISION OBSTRUCTION (INCL. BLINDED BY SUN), WET',
                                        }
#
v_exhaust = 'Vehicle Exhaust System'
v_exhaust_wet='Vehicle Exhaust System'

v_exhaust_variants={
 'V EXHAUST SYSTEM|R OTHER ROAD, V WIPERS|W OTHER ENVIRONMENTAL',
 'V EXHAUST SYSTEM|R OTHER ROAD, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
 'N/A, V EXHAUST SYSTEM|R OTHER ROAD',
}
v_exhaust_wet_variants ={
  'N/A, V EXHAUST SYSTEM|R OTHER ROAD, WET',
  'RAIN, SNOW, V EXHAUST SYSTEM|R OTHER ROAD, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'RAIN, SNOW, V EXHAUST SYSTEM|R OTHER ROAD, WET',
  'RAIN, SNOW, V EXHAUST SYSTEM|R OTHER ROAD',
  }
#
snow = 'Snowy Conditions'
snow_variants={
  'N/A, RAIN, SNOW',
  'RAIN, SNOW',
  'RAIN, SNOW, WET',
  }
#

traffic_controller = 'Traffic Controller Device Inoperative'
traffic_controller_wet= 'Traffic Controller Device Inoperative- Wet Conditions'

traffic_controller_variants ={
  'N/A, TRAFFIC CONTROL DEVICE INOPERATIVE',
  'RAIN, SNOW, TRAFFIC CONTROL DEVICE INOPERATIVE',
  }
traffic_controller_wet_variants={
  'N/A, TRAFFIC CONTROL DEVICE INOPERATIVE, WET',
  'RAIN, SNOW, TRAFFIC CONTROL DEVICE INOPERATIVE, WET',
  }


#
other_environmental_wet = 'Other Environmental Factors- Wet Conditions'
other_environmental_wet_variants= {
  'V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'RAIN, SNOW, V WIPERS|W OTHER ENVIRONMENTAL, WET',
  'N/A, RAIN, SNOW, V WIPERS|W OTHER ENVIRONMENTAL',

  }
#
road_under_construction = 'Road Under Construction/Maintenance'
road_under_construction_wet = 'Road Under Construction/Maintenance- Wet Conditions'

road_under_construction_variants = {
  'N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE, RUTS, HOLES, BUMPS, WORN, TRAVEL-POLISHED SURFACE',
  'N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE, RUTS, HOLES, BUMPS',
  'N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE, TRAFFIC CONTROL DEVICE INOPERATIVE',
  'N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE',
  'ROAD UNDER CONSTRUCTION/MAINTENANCE, V WIPERS|W OTHER ENVIRONMENTAL',
  'N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE, V EXHAUST SYSTEM|R OTHER ROAD',
  'RAIN, SNOW, ROAD UNDER CONSTRUCTION/MAINTENANCE, SEVERE CROSSWINDS',
  }
road_under_construction_wet_variants = {
  'RAIN, SNOW, ROAD UNDER CONSTRUCTION/MAINTENANCE, WET',
  'RAIN, SNOW, ROAD UNDER CONSTRUCTION/MAINTENANCE, RUTS, HOLES, BUMPS',
  'N/A, ROAD UNDER CONSTRUCTION/MAINTENANCE, WET',
  'RAIN, SNOW, ROAD UNDER CONSTRUCTION/MAINTENANCE, RUTS, HOLES, BUMPS, WET',
  'RAIN, SNOW, ROAD UNDER CONSTRUCTION/MAINTENANCE',
  'ROAD UNDER CONSTRUCTION/MAINTENANCE, VISION OBSTRUCTION (INCL. BLINDED BY SUN)',
  'ROAD UNDER CONSTRUCTION/MAINTENANCE, SEVERE CROSSWINDS',
  'N/A, NON-HIGHWAY WORK, ROAD UNDER CONSTRUCTION/MAINTENANCE',
  }
#
Disregarded_RoadMarkings_TrafficSign ='Disregarded Other Road Markings, Disregarded Other Traffic Sign'
Disregarded_RoadMarkings_TrafficSign_variants ={
  'Disregarded Other Road Markings',
  'Disregarded Other Road Markings, Other Improper Action',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Other Improper Action, Ran Off Roadway, Ran Red Light',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Improper Passing',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Improper Turn',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Passing, Ran Red Light',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Improper Passing, Other Improper Action',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Turn',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Passing, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Ran Off Roadway',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions, Wrong Side, Wrong Way',
  'Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Yield Right-of-Way',
      }
#
DisregardedRoadMarkings_FailedKeepProperLane = 'Disregarded Other Road Markings, Failed to Keep in Proper Lane'
DisregardedRoadMarkings_FailedKeepProperLane_variants = {
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Improper Passing, Operated Motor Vehicle in Reckless or Aggressive Manner, Wrong Side',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Improper Turn, Ran Off Roadway',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Improper Turn',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Improper Passing',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Failed to Yield',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Other Improper Action',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Wrong Side',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Followed Too Closely, Improper Passing, Other Improper Action, Wrong Side, Wrong Way',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Other Improper Action, Ran Off Roadway, Too Fast For Conditions',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Followed Too Closely, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Ran Off Roadway',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Improper Passing, Wrong Side',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway, Too Fast For Conditions',
  'Disregarded Other Road Markings, Failed to Keep in Proper Lane, Too Fast For Conditions, Wrong Side',
  }
#
DisregardedRoadMarkings_FailedYield_RightofWay = 'Disregarded Other Road Markings, Failed to Yield Right-of-Way'
DisregardedRoadMarkings_FailedYield_RightofWay_variants = {
  'Disregarded Other Road Markings, Failed to Yield Right-of-Way, Improper Turn',
 'Disregarded Other Road Markings, Failed to Yield Right-of-Way',
 'Disregarded Other Road Markings, Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
 'Disregarded Other Road Markings, Failed to Yield Right-of-Way, Improper Passing, Improper Turn',
 'Disregarded Other Road Markings, Failed to Yield Right-of-Way, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
}
#
DisregardedRoadMarking_RanOffRoadway ='Disregarded Other Road Markings, Ran Off Roadway'
DisregardedRoadMarking_RanOffRoadway_variants ={
  'Disregarded Other Road Markings, Other Improper Action, Ran Off Roadway, Wrong Side, Wrong Way',
  'Disregarded Other Road Markings, Ran Off Roadway',
  }
#
DisregardedRoadMarkings__OperatedInattentive='Disregarded Other Road Markings, Operated Motor Vehicle in Inattentive'
DisregardedRoadMarkings__OperatedInattentive_variants ={
  'Disregarded Other Road Markings, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Wrong Way',
  'Disregarded Other Road Markings, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Wrong Side',
  'Disregarded Other Road Markings, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Disregarded Other Road Markings, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
  'Disregarded Other Road Markings, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
  }
#
DisregardedRoadMarkings_ImproperPassing ='Disregarded Other Road Markings, Improper Passing'
DisregardedRoadMarkings_ImproperPassing_variants ={
  'Disregarded Other Road Markings, Improper Passing, Wrong Side',
  'Disregarded Other Road Markings, Improper Passing, Other Improper Action',
  }
#
RanOff_Roadway = 'Ran Off Roadway'
RanOff_Roadway_variants={
  'Ran Off Roadway',
  'Other Improper Action, Ran Off Roadway',
  }
#
RanOffRoadway_RanStopSign ='Ran Off Roadway, Ran Stop Sign'
RanOffRoadway_RanStopSign_variants = {
  'Ran Off Roadway, Ran Stop Sign',
  'Ran Off Roadway, Ran Stop Sign, Too Fast For Conditions',
  }
#
RanOffRoadway_SwervedAvoided_DueWind ='Ran Off Roadway, Swerved/Avoided Due to Wind'
RanOffRoadway_SwervedAvoided_DueWind_variants ={
  'Ran Off Roadway, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
  'Ran Off Roadway, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
  'Other Improper Action, Ran Off Roadway, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
  }
#
RanOffRoadway_TooFast = 'Ran Off Roadway, Too Fast For Conditions'
RanOffRoadway_TooFast_variants ={
  'Ran Off Roadway, Too Fast For Conditions',
  'Other Improper Action, Ran Off Roadway, Too Fast For Conditions',
  }
#
OverCorrecting_OverSteering_RanOffRoadway = 'Over-Correcting/Over-Steering, Ran Off Roadway'
OverCorrecting_OverSteering_RanOffRoadway_variants ={
  'Over-Correcting/Over-Steering, Ran Off Roadway, Wrong Side',
  'Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions',
  'Over-Correcting/Over-Steering, Ran Off Roadway',
  'Over-Correcting/Over-Steering, Ran Off Roadway, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
  'Over-Correcting/Over-Steering, Ran Off Roadway, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
  }
#
OverCorrecting_OverSteering_SwervedAvoided_DueWind ='Over-Correcting/Over-Steering, Swerved/Avoided Due to Wind'
OverCorrecting_OverSteering_SwervedAvoided_DueWind_variants ={
  'Over-Correcting/Over-Steering, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
  'Over-Correcting/Over-Steering, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
  }
#
FollowedTooClosely_OperatedInattentive ='Followed Too Closely, Operated Motor Vehicle in Inattentive'
FollowedTooClosely_OperatedInattentive_variants ={
  'Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action', 'Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Too Fast For Conditions',
  'Followed Too Closely, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Followed Too Closely, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
  'Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions',
  'Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions',
  'Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner',
  }
#
FollowedTooClosely_SwervedAvoidedDueWind ='Followed Too Closely, Swerved or Avoided Due to Wind'
FollowedTooClosely_SwervedAvoidedDueWind_variants ={
  'Followed Too Closely, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
  'Followed Too Closely, Other Improper Action, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
  'Followed Too Closely, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
  }
#
FollowedTooClosely_ImproperPassing = 'Followed Too Closely, Improper Passing'
FollowedTooClosely_ImproperPassing_variants = {
  'Followed Too Closely, Improper Passing, Other Improper Action',
  'Followed Too Closely, Improper Passing, Too Fast For Conditions',
  'Followed Too Closely, Improper Passing, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  }
#
FollowedTooClosely_OperatedRecklessAggressive = 'Followed Too Closely, Operated Motor Vehicle in Reckless or Aggressive Manner'
FollowedTooClosely_OperatedRecklessAggressive_variants ={
  'Followed Too Closely, Operated Motor Vehicle in Reckless or Aggressive Manner',
  'Followed Too Closely, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  }
#
FollowedTooClosely_TooFastConditions = 'Followed Too Closely, Too Fast For Conditions'
FollowedTooClosely_TooFastConditions_variants = {
  'Followed Too Closely, Too Fast For Conditions',
  'Followed Too Closely, Other Improper Action, Too Fast For Conditions',
  'Disregarded Other Road Markings, Followed Too Closely, Too Fast For Conditions',
  }

#
FollowedTooClosely = 'Followed Too Closely'
FollowedTooClosely_variants =  {
  'Followed Too Closely',
  'Followed Too Closely, Other Improper Action',
  }
#

FollowedTooClosely_RanRedLight = 'Followed Too Closely, Ran Red Light'
FollowedTooClosely_RanRedLight_variants ={
  'Followed Too Closely, Ran Red Light, Too Fast For Conditions',
  'Followed Too Closely, Ran Red Light',
  'Followed Too Closely, Ran Off Roadway',
  'Followed Too Closely, Improper Turn',
  'Followed Too Closely, Wrong Side',
  'Followed Too Closely, Over-Correcting/Over-Steering',
  'Followed Too Closely, Improper Passing',
  }
#
RanRedLight_TooFastConditions ='Ran Red Light, Too Fast For Conditions'
RanRedLight_TooFastConditions_variants = {
  'Other Improper Action, Ran Red Light, Too Fast For Conditions',
  'Ran Red Light, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
  'Ran Red Light, Too Fast For Conditions',
  }
#
FailedKeepProperLane_OperatedInattentive ='Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive'
FailedKeepProperLane_OperatedInattentive_variants ={
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions, Wrong Side, Wrong Way',
  'Failed to Keep in Proper Lane, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway, Wrong Side',
  'Failed to Keep in Proper Lane, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions, Wrong Side, Wrong Way',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Followed Too Closely, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering, Ran Off Roadway, Wrong Side',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Ran Off Roadway',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway, Wrong Side',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions, Wrong Side',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner',
  #Inattentive
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway, Wrong Side, Wrong Way',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway, Too Fast For Conditions, Wrong Side',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Wrong Side',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions, Wrong Side, Wrong Way',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Over-Correcting/Over-Steering',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Wrong Side',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  }
#
FailedKeepProperLane_FailedYieldRightofWay = 'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way'
FailedKeepProperLane_FailedYieldRightofWay_variants ={
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Passing, Wrong Side',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Followed Too Closely, Improper Passing',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Backing',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Followed Too Closely, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Ran Stop Sign',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Other Improper Action',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Wrong Side',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Turn, Other Improper Action',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Too Fast For Conditions, Wrong Side, Wrong Way',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Operated Motor Vehicle in Reckless or Aggressive Manner',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Turn',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Passing',
  'Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Turn, Ran Red Light',
  }
#

FailedKeepProperLane_FollowedTooClosely = 'Failed to Keep in Proper Lane, Followed Too Closely'
FailedKeepProperLane_FollowedTooClosely_variants ={
  'Failed to Keep in Proper Lane, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Followed Too Closely, Improper Passing, Other Improper Action, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions, Wrong Side',
  'Failed to Keep in Proper Lane, Followed Too Closely, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Wrong Side',
  'Failed to Keep in Proper Lane, Followed Too Closely',
  'Failed to Keep in Proper Lane, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
  'Failed to Keep in Proper Lane, Followed Too Closely, Over-Correcting/Over-Steering',
  'Failed to Keep in Proper Lane, Followed Too Closely, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions, Wrong Side',
  'Failed to Keep in Proper Lane, Followed Too Closely, Other Improper Action',
  'Failed to Keep in Proper Lane, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Followed Too Closely, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
  'Failed to Keep in Proper Lane, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
  }
#
FailedKeepProperLane_OperatedReckless = 'Failed to Keep in Proper Lane, Operated Motor Vehicle in Reckless'
FailedKeepProperLane_OperatedReckless_variants ={
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Red Light, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering, Too Fast For Conditions, Wrong Side',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in Reckless or Aggressive Manner',
  'Failed to Keep in Proper Lane, Operated Motor Vehicle in, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway',
  }
#
FailedKeepProperLane_ImproperPassing = 'Failed to Keep in Proper Lane, Improper Passing'
FailedKeepProperLane_ImproperPassing_variants ={
  'Failed to Keep in Proper Lane, Improper Passing, Operated Motor Vehicle in Reckless or Aggressive Manner',
  'Failed to Keep in Proper Lane, Improper Passing, Improper Turn, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Improper Passing, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Failed to Keep in Proper Lane, Improper Passing, Improper Turn',
  'Failed to Keep in Proper Lane, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
  'Failed to Keep in Proper Lane, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Improper Passing, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
  'Failed to Keep in Proper Lane, Improper Passing',
  }
#
FailedKeepProperLane_ImproperTurn ='Failed to Keep in Proper Lane, Improper Turn'
FailedKeepProperLane_ImproperTurn_variants = {
  'Failed to Keep in Proper Lane, Improper Turn, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Improper Turn, Ran Off Roadway',
  'Failed to Keep in Proper Lane, Improper Turn, Other Improper Action',
  'Failed to Keep in Proper Lane, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Failed to Keep in Proper Lane, Improper Turn, Over-Correcting/Over-Steering, Wrong Way',
  'Failed to Keep in Proper Lane, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering',
  'Failed to Keep in Proper Lane, Improper Turn, Over-Correcting/Over-Steering',
  'Failed to Keep in Proper Lane, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering',
  'Failed to Keep in Proper Lane, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions, Wrong Side, Wrong Way',
  'Failed to Keep in Proper Lane, Improper Turn',
  'Failed to Keep in Proper Lane, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway',
  'Failed to Keep in Proper Lane, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
  'Failed to Keep in Proper Lane, Improper Turn, Other Improper Action, Ran Off Roadway',
  }
#
FailedKeepProperLane_OverCorrectingSteering = 'Failed to Keep in Proper Lane, Over-Correcting/Over-Steering'
FailedKeepProperLane_OverCorrectingSteering_variants  = {
  'Failed to Keep in Proper Lane, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Over-Correcting/Over-Steering, Ran Off Roadway',
  'Failed to Keep in Proper Lane, Over-Correcting/Over-Steering, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Over-Correcting/Over-Steering',
  'Failed to Keep in Proper Lane, Other Improper Action, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions',
  }
#
FailedKeepProperLane_RanOffRoadway ='Failed to Keep in Proper Lane, Ran Off Roadway'
FailedKeepProperLane_RanOffRoadway_variants = {
  'Failed to Keep in Proper Lane, Ran Off Roadway, Too Fast For Conditions, Wrong Way',
  'Failed to Keep in Proper Lane, Ran Off Roadway, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Ran Off Roadway, Too Fast For Conditions, Wrong Side',
  'Failed to Keep in Proper Lane, Ran Off Roadway',
  'Failed to Keep in Proper Lane, Other Improper Action, Ran Off Roadway',
  'Failed to Keep in Proper Lane, Other Improper Action, Ran Off Roadway, Too Fast For Conditions',
  }
#
FailedKeepProperLane_TooFastConditions = 'Failed to Keep in Proper Lane, Too Fast For Conditions'
FailedKeepProperLane_TooFastConditions_variants = {
  'Failed to Keep in Proper Lane, Too Fast For Conditions, Wrong Way',
  'Failed to Keep in Proper Lane, Too Fast For Conditions, Wrong Side',
  'Failed to Keep in Proper Lane, Too Fast For Conditions',
  'Failed to Keep in Proper Lane, Other Improper Action, Too Fast For Conditions',
  }
#
FailedKeepProperLane_WrongWay ='Failed to Keep in Proper Lane, Wrong Side/Wrong Way'
FailedKeepProperLane_WrongWay_variants = {
  'Failed to Keep in Proper Lane, Wrong Way',
  'Failed to Keep in Proper Lane, Wrong Side, Wrong Way',
  'Failed to Keep in Proper Lane, Wrong Side',
  'Failed to Keep in Proper Lane, Other Improper Action, Wrong Side',
  }
#
FailedKeep_ProperLane = 'Failed to Keep in Proper Lane '
FailedKeep_ProperLane_variants = {
  'Failed to Keep in Proper Lane',
  'Failed to Keep in Proper Lane, Other Improper Action',
  }
#
DisregardedTrafficSign_FailedYieldRightofWay = 'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way'
DisregardedTrafficSign_FailedYieldRightofWay_variants ={
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Stop Sign',
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Red Light',
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Ran Stop Sign',
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Other Improper Action',
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Ran Red Light, Too Fast For Conditions',
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way, Improper Turn',
  'Disregarded Other Traffic Sign, Failed to Yield Right-of-Way',
  }
#
DisregardedTrafficSign_FailedKeepProperLane ='Disregarded Other Traffic Sign, Failed to Keep in Proper Lane'
DisregardedTrafficSign_FailedKeepProperLane_variants ={
  'Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Passing, Improper Turn, Other Improper Action, Wrong Side, Wrong Way',
  'Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Red Light',
  'Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Over-Correcting/Over-Steering, Ran Off Roadway',
  'Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Disregarded Other Traffic Sign, Failed to Keep in Proper Lane',
  'Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Improper Turn',
  }

#
DisregardedTrafficSign_RanRedLight = 'Disregarded Other Traffic Sign, Ran Red Light'
DisregardedTrafficSign_RanRedLight_variants = {
  'Disregarded Other Traffic Sign, Ran Red Light',
  'Disregarded Other Traffic Sign, Ran Red Light, Too Fast For Conditions',
  'Disregarded Other Traffic Sign, Ran Red Light, Too Fast For Conditions',
  }
#
DisregardedTrafficSign_OperatedInattentive = 'Disregarded Other Traffic Sign, Operated Motor Vehicle in Inattentive'
DisregardedTrafficSign_OperatedInattentive_variants ={
  'Disregarded Other Traffic Sign, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions',
  'Disregarded Other Traffic Sign, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway',
  'Disregarded Other Traffic Sign, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway',
  'Disregarded Other Traffic Sign, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Red Light',
  'Disregarded Other Traffic Sign, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Red Light, Too Fast For Conditions',
  'Disregarded Other Traffic Sign, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner',
  'Disregarded Other Traffic Sign, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Over-Correcting/Over-Steering, Ran Red Light',
  'Disregarded Other Traffic Sign, Operated Motor Vehicle in Iattentive, Careless, Negligent, or Erratic Manner',
  }
#
DisregardedTrafficSign_ImproperTurn ='Disregarded Other Traffic Sign, Improper Turn'
DisregardedTrafficSign_ImproperTurn_variants = {
  'Disregarded Other Traffic Sign, Improper Turn',
  'Disregarded Other Traffic Sign, Improper Turn, Other Improper Action',
  'Disregarded Other Traffic Sign, Improper Turn, Wrong Way',
  }
#

DisregardedOther_TrafficSign = 'Disregarded Other Traffic Sign'
DisregardedOther_TrafficSign_variants={
  'Disregarded Other Traffic Sign',
  'Disregarded Other Traffic Sign, Other Improper Action',
  }
#
OperatedInattentiveCarelessNegligentErratic_OperatedRecklessAggressive ='Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner'
OperatedInattentiveCarelessNegligentErratic_OperatedRecklessAggressive_variants ={
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Over-Correcting/Over-Steering',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Red Light, Too Fast For Conditions',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering, Ran Off Roadway, Wrong Side',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions, Wrong Side',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Over-Correcting/Over-Steering, Ran Off Roadway',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Red Light',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
  'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Wrong Way',
  }
#
OperatedInattentiveCarelessNegligentErratic_RanOffRoadway ='Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway'
OperatedInattentiveCarelessNegligentErratic_RanOffRoadway_variants = {
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway, Wrong Side, Wrong Way',
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway, Ran Red Light',
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway, Too Fast For Conditions',
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway, Ran Stop Sign',
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway',
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Ran Off Roadway, Too Fast For Conditions',
 }
#
OperatedInattentiveCarelessNegligentErratic_OverCorrectingSteering ='Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering'
OperatedInattentiveCarelessNegligentErratic_OverCorrectingSteering_variants ={
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions',
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering, Ran Off Roadway',
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering',
 'Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Over-Correcting/Over-Steering',
 }
#
DisregardOfficerDirections_DisregardedOtherRoadMarkings='Disregard Officer Directions, Disregarded Other Road Markings'
DisregardOfficerDirections_DisregardedOtherRoadMarkings_variants = {
 'Disregard Officer Directions, Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Failed to Yield Right-of-Way, Followed Too Closely, Improper Passing, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions, Wrong Side, Wrong Way',
 'Disregard Officer Directions, Disregarded Other Road Markings, Failed to Keep in Proper Lane, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions, Wrong Way',
 'Disregard Officer Directions, Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway, Too Fast For Conditions',
 'Disregard Officer Directions, Disregarded Other Road Markings, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Other Improper Action',
 }
#
DisregardOfficerDirections_OtherTrafficSign ='Disregard Officer Directions, Disregarded Other Traffic Sign'
DisregardOfficerDirections_OtherTrafficSign_variants = {
  'Disregard Officer Directions, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
  'Disregard Officer Directions, Disregarded Other Traffic Sign, Failed to Keep in Proper Lane, Followed Too Closely, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions',
  }
#
DisregardOfficerDirections_OperatedRecklessAggressive ='Disregard Officer Directions, Operated Motor Vehicle in Reckless or Aggressive Manner'
DisregardOfficerDirections_OperatedRecklessAggressive_variants ={
 'Disregard Officer Directions, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
 'Disregard Officer Directions, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway',
 'Disregard Officer Directions, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Red Light',
 }
#
DisregardOfficerDirections_ImproperBacking ='Disregard Officer Directions, Improper Backing'
DisregardOfficerDirections_ImproperBacking_variants ={
  'Disregard Officer Directions, Improper Backing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
  'Disregard Officer Directions, Improper Backing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway, Too Fast For Conditions',
  }
#
DisregardOfficerDirections_OperatedInattentive ='Disregard Officer Directions, Operated Motor Vehicle in Inattentive'
DisregardOfficerDirections_OperatedInattentive_variants ={
 'Disregard Officer Directions, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions',
 'Disregard Officer Directions, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway',
 'Disregard Officer Directions, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
 }
#
FailedYieldRightofWay_OperatedInattentive ='Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive'
FailedYieldRightofWay_OperatedInattentive_variants ={
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Stop Sign',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Red Light',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Red Light',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Too Fast For Conditions',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Ran Red Light',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Red Light, Too Fast For Conditions',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Too Fast For Conditions',
 }
#
FailedYieldRightofWay_ImproperPassing = 'Failed to Yield Right-of-Way, Improper Passing'
FailedYieldRightofWay_ImproperPassing_variants = {
 'Failed to Yield Right-of-Way, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway',
 'Failed to Yield Right-of-Way, Improper Passing, Too Fast For Conditions',
 'Failed to Yield Right-of-Way, Improper Passing, Other Improper Action',
 'Failed to Yield Right-of-Way, Improper Passing',
 'Failed to Yield Right-of-Way, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
 'Failed to Yield Right-of-Way, Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
 }
#
FailedYieldRightofWay_OperatedRecklessAggressive = 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Reckless or Aggressive Manner'
FailedYieldRightofWay_OperatedRecklessAggressive_variants = {
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Stop Sign, Too Fast For Conditions',
 'Failed to Yield Right-of-Way, Operated Motor Vehicle in Reckless or Aggressive Manner',
 }
#
FailedYieldRightofWay_FollowedTooClosely ='Failed to Yield Right-of-Way, Followed Too Closely'
FailedYieldRightofWay_FollowedTooClosely_variants ={
 'Failed to Yield Right-of-Way, Followed Too Closely, Too Fast For Conditions',
 'Failed to Yield Right-of-Way, Followed Too Closely',
 'Failed to Yield Right-of-Way, Followed Too Closely, Improper Turn',
 'Failed to Yield Right-of-Way, Followed Too Closely, Ran Stop Sign',
 'Failed to Yield Right-of-Way, Followed Too Closely, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
 }
#
FailedYieldRightofWay_RanStopSign ='Failed to Yield Right-of-Way, Ran Stop Sign'
FailedYieldRightofWay_RanStopSign_variants ={
 'Failed to Yield Right-of-Way, Ran Stop Sign, Too Fast For Conditions',
 'Failed to Yield Right-of-Way, Ran Stop Sign',
 'Failed to Yield Right-of-Way, Other Improper Action, Ran Stop Sign, Too Fast For Conditions',
 }
#
FailedYieldRightofWay_ImproperTurn='Failed to Yield Right-of-Way, Improper Turn'
FailedYieldRightofWay_ImproperTurn_variants ={
 'Failed to Yield Right-of-Way, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
 'Failed to Yield Right-of-Way, Improper Turn, Ran Stop Sign',
 'Failed to Yield Right-of-Way, Improper Turn, Ran Off Roadway',
 'Failed to Yield Right-of-Way, Improper Turn',
 'Failed to Yield Right-of-Way, Improper Turn, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
 'Failed to Yield Right-of-Way, Improper Turn, Other Improper Action',
 'Failed to Yield Right-of-Way, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Stop Sign',
 'Failed to Yield Right-of-Way, Improper Turn, Ran Red Light, Too Fast For Conditions',
 'Failed to Yield Right-of-Way, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Ran Red Light',
 'Failed to Yield Right-of-Way, Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
 }
#
FailedYieldRightofWay_ImproperBacking ='Failed to Yield Right-of-Way, Improper Backing'
FailedYieldRightofWay_ImproperBacking_variants ={
 'Failed to Yield Right-of-Way, Improper Backing, Ran Red Light',
 'Failed to Yield Right-of-Way, Improper Backing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action',
 'Failed to Yield Right-of-Way, Improper Backing',
 }
#
FailedYieldRightofWay_RanRedLight ='Failed to Yield Right-of-Way, Ran Red Light'
FailedYieldRightofWay_RanRedLight_variants ={
 'Failed to Yield Right-of-Way, Ran Red Light',
 'Failed to Yield Right-of-Way, Ran Red Light, Too Fast For Conditions',
 'Failed to Yield Right-of-Way, Other Improper Action, Ran Red Light',
 }
#
Failed_YieldRightofWay ='Failed to Yield Right-of-Way'
Failed_YieldRightofWay_variants = {
 'Failed to Yield Right-of-Way',
 'Failed to Yield Right-of-Way, Other Improper Action',
 }
#
OperatedRecklessAggressiver_RanOffRoadway='Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway'
OperatedRecklessAggressiver_RanOffRoadway_variants ={
 'Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway, Too Fast For Conditions',
 'Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Ran Off Roadway, Too Fast For Conditions',
 'Operated Motor Vehicle in Reckless or Aggressive Manner, Ran Off Roadway',
 }
#
ImproperTurn_OperatedInattentiveCarelessNegligent ='Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent'
ImproperTurn_OperatedInattentiveCarelessNegligent_variants ={
 'Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
 'Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions, Wrong Side',
 'Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
 'Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering, Ran Off Roadway, Too Fast For Conditions',
 'Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering',
 'Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner',
 'Improper Turn, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
 }
#
ImproperTurn_OverCorrectingSteering ='Improper Turn, Over-Correcting/Over-Steering'
ImproperTurn_OverCorrectingSteering_variants ={
 'Improper Turn, Over-Correcting/Over-Steering',
 'Improper Turn, Over-Correcting/Over-Steering, Too Fast For Conditions',
 'Improper Turn, Over-Correcting/Over-Steering, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
 }
#
ImproperBacking_OperatedInattentiveCareless ='Improper Backing, Operated Motor Vehicle in Inattentive, Careless'
ImproperBacking_OperatedInattentiveCareless_variants= {
 'Improper Backing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner',
 'Improper Backing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
 'Improper Backing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Stop Sign',
 'Improper Backing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Other Improper Action, Over-Correcting/Over-Steering',
 'Improper Backing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Other Improper Action',
 'Improper Backing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Ran Off Roadway',
 }
#
ImproperPassing_OperatedInattentiveCarelessNegligentErratic ='Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner'
ImproperPassing_OperatedInattentiveCarelessNegligentErratic_variants ={
 'Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Over-Correcting/Over-Steering',
 'Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions, Wrong Way',
 'Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Too Fast For Conditions',
 'Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Operated Motor Vehicle in Reckless or Aggressive Manner',
 'Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner',
 'Improper Passing, Operated Motor Vehicle in Inattentive, Careless, Negligent, or Erratic Manner, Wrong Side',
 }
#
ImproperPassing_OperatedRecklessAggressive  ='Improper Passing, Operated Motor Vehicle in Reckless or Aggressive Manner'
ImproperPassing_OperatedRecklessAggressive_variants ={
 'Improper Passing, Operated Motor Vehicle in Reckless or Aggressive Manner',
 'Improper Passing, Operated Motor Vehicle in Reckless or Aggressive Manner, Over-Correcting/Over-Steering, Too Fast For Conditions',
 'Improper Passing, Operated Motor Vehicle in Reckless or Aggressive Manner, Too Fast For Conditions',
 }
#
TooFast_ForConditions ='Too Fast For Conditions'
TooFast_ForConditions_variants ={
 'Too Fast For Conditions, Wrong Side',
 'Too Fast For Conditions',
 'Other Improper Action, Too Fast For Conditions',
 'Disregarded Other Road Markings, Too Fast For Conditions, Wrong Side',
 }
#
Swerved_DueWind_SlipperySurface ='Swerved or Avoided Due to Wind, Slippery Surface'
Swerved_DueWind_SlipperySurface_variants = {
 'Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Too Fast For Conditions',
 'Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc, Wrong Side',
 'Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
 'Other Improper Action, Swerved or Avoided Due to Wind, Slippery Surface, Motor Vehicle, Object, Non-Motorist in Roadway, etc',
 }




# value - variants
replacements_circumstance = {
  **{key: animal for key in animal_variants},
  **{key: animal_wet for key in animal_wet_variants },
	**{key: blowing_sand for key in blowing_sand_variants},
  **{key: blowing_sand_wet for key in blowing_sand_wet_variants },
  **{key: icy_snow_covered for key in icy_snow_covered_variants },
  **{key: sleet_hail for key in sleet_hail_variants },
  **{key: debris_obstruction for key in debris_obstruction_variants },
  **{key: debris_obstruction_wet for key in debris_obstruction_wet_variants },
  **{key: ruts for key in ruts_variants },
  **{key: ruts_wet for key in ruts_wet_variants},

  **{key: backup_nonrecurring_incident for key in backup_nonrecurring_incident_variants},
  **{key: backup_nonrecurring_incident_wet for key in backup_nonrecurring_incident_wet_variants},

  **{key: backup_prior_crash for key in backup_prior_crash_variants},
  **{key: backup_prior_crash_wet for key in backup_prior_crash_wet_variants},
  **{key: backup_regular_congestion for key in backup_regular_congestion_variants},
  **{key: backup_regular_congestion_wet for key in backup_regular_congestion_wet_variants},

  **{key: wind for key in wind_variants},
  **{key: wind_wet for key in wind_wet_variants},
  **{key: polished for key in polished_variants},
  **{key: polished_wet for key in polished_wet_variants},
  **{key: shoulder for key in shoulder_variants},
  **{key: shoulder_wet for key in shoulder_wet_variants},
  **{key: smog for key in smog_variants},
  **{key: vision_obstruction for key in vision_obstruction_variants},
  **{key: vision_obstruction_wet for key in vision_obstruction_wet_variants},
  **{key: physical_obstruction for key in physical_obstruction_variants},
  **{key: physical_obstruction_wet for key in physical_obstruction_wet_variants},

  **{key: v_exhaust for key in v_exhaust_variants},
  **{key: v_exhaust_wet for key in v_exhaust_wet_variants},
  **{key: traffic_controller for key in traffic_controller_variants},
  **{key: traffic_controller_wet for key in traffic_controller_wet_variants},
  **{key: snow for key in snow_variants },
  **{key: other_environmental_wet for key in other_environmental_wet_variants},
  **{key: road_under_construction for key in road_under_construction_variants},
  **{key: road_under_construction_wet for key in road_under_construction_wet_variants},
  #non caps below
  **{key: Disregarded_RoadMarkings_TrafficSign for key in Disregarded_RoadMarkings_TrafficSign_variants},
  **{key: DisregardedRoadMarkings_FailedKeepProperLane for key in DisregardedRoadMarkings_FailedKeepProperLane_variants},
  **{key: DisregardedRoadMarkings_FailedYield_RightofWay for key in DisregardedRoadMarkings_FailedYield_RightofWay_variants},
  **{key: DisregardedRoadMarking_RanOffRoadway for key in DisregardedRoadMarking_RanOffRoadway_variants},
  **{key: DisregardedRoadMarkings__OperatedInattentive for key in DisregardedRoadMarkings__OperatedInattentive_variants},  **{key: DisregardedRoadMarkings_ImproperPassing for key in DisregardedRoadMarkings_ImproperPassing_variants},
  **{key: DisregardedRoadMarkings_ImproperPassing for key in DisregardedRoadMarkings_ImproperPassing_variants},
  **{key: RanOff_Roadway for key in RanOff_Roadway_variants},
  **{key: RanOffRoadway_RanStopSign for key in RanOffRoadway_RanStopSign_variants},
  **{key: RanOffRoadway_SwervedAvoided_DueWind for key in RanOffRoadway_SwervedAvoided_DueWind_variants},
  **{key: RanOffRoadway_TooFast for key in RanOffRoadway_TooFast_variants},
  **{key: OverCorrecting_OverSteering_RanOffRoadway for key in OverCorrecting_OverSteering_RanOffRoadway_variants},
  **{key: OverCorrecting_OverSteering_SwervedAvoided_DueWind for key in OverCorrecting_OverSteering_SwervedAvoided_DueWind_variants},
  **{key: FollowedTooClosely_OperatedInattentive for key in FollowedTooClosely_OperatedInattentive_variants},
  **{key: FollowedTooClosely_SwervedAvoidedDueWind for key in FollowedTooClosely_SwervedAvoidedDueWind_variants},
  **{key: FollowedTooClosely_ImproperPassing for key in FollowedTooClosely_ImproperPassing_variants},
  **{key: FollowedTooClosely_OperatedRecklessAggressive for key in FollowedTooClosely_OperatedRecklessAggressive_variants},
  **{key: FollowedTooClosely for key in FollowedTooClosely_variants},
  **{key: FollowedTooClosely_RanRedLight for key in FollowedTooClosely_RanRedLight_variants},
  **{key: RanRedLight_TooFastConditions for key in RanRedLight_TooFastConditions_variants},
  **{key: FailedKeepProperLane_OperatedInattentive for key in FailedKeepProperLane_OperatedInattentive_variants},
  **{key: FailedKeepProperLane_FailedYieldRightofWay for key in FailedKeepProperLane_FailedYieldRightofWay_variants},
  **{key: FailedKeepProperLane_FollowedTooClosely for key in FailedKeepProperLane_FollowedTooClosely_variants},
  **{key: FailedKeepProperLane_OperatedReckless for key in FailedKeepProperLane_OperatedReckless_variants},
  **{key: FailedKeepProperLane_ImproperPassing for key in FailedKeepProperLane_ImproperPassing_variants},
  **{key: FailedKeepProperLane_ImproperTurn for key in FailedKeepProperLane_ImproperTurn_variants},
  **{key: FailedKeepProperLane_OverCorrectingSteering for key in FailedKeepProperLane_OverCorrectingSteering_variants},
#
  **{key: FailedKeepProperLane_RanOffRoadway for key in FailedKeepProperLane_RanOffRoadway_variants},
  **{key: FailedKeepProperLane_TooFastConditions for key in FailedKeepProperLane_TooFastConditions_variants},
  **{key: FailedKeepProperLane_WrongWay for key in FailedKeepProperLane_WrongWay_variants},
  **{key: FailedKeep_ProperLane for key in FailedKeep_ProperLane_variants},
  **{key: DisregardedTrafficSign_FailedYieldRightofWay for key in DisregardedTrafficSign_FailedYieldRightofWay_variants},
  **{key: DisregardedTrafficSign_FailedKeepProperLane for key in DisregardedTrafficSign_FailedKeepProperLane_variants},
  **{key: DisregardedTrafficSign_RanRedLight for key in DisregardedTrafficSign_RanRedLight_variants},
  **{key: DisregardedTrafficSign_OperatedInattentive for key in DisregardedTrafficSign_OperatedInattentive_variants},
  **{key: DisregardedTrafficSign_ImproperTurn for key in DisregardedTrafficSign_ImproperTurn_variants},
  **{key: DisregardedOther_TrafficSign for key in DisregardedOther_TrafficSign_variants},
  **{key: OperatedInattentiveCarelessNegligentErratic_OperatedRecklessAggressive for key in OperatedInattentiveCarelessNegligentErratic_OperatedRecklessAggressive_variants},
  **{key: OperatedInattentiveCarelessNegligentErratic_RanOffRoadway for key in OperatedInattentiveCarelessNegligentErratic_RanOffRoadway_variants},
  **{key: OperatedInattentiveCarelessNegligentErratic_OverCorrectingSteering for key in OperatedInattentiveCarelessNegligentErratic_OverCorrectingSteering_variants},
  **{key: DisregardOfficerDirections_DisregardedOtherRoadMarkings for key in DisregardOfficerDirections_DisregardedOtherRoadMarkings_variants},
  **{key: DisregardOfficerDirections_OtherTrafficSign for key in DisregardOfficerDirections_OtherTrafficSign_variants},
  **{key: DisregardOfficerDirections_OperatedRecklessAggressive for key in DisregardOfficerDirections_OperatedRecklessAggressive_variants},
  **{key: DisregardOfficerDirections_ImproperBacking for key in DisregardOfficerDirections_ImproperBacking_variants},
  **{key: DisregardOfficerDirections_OperatedInattentive for key in DisregardOfficerDirections_OperatedInattentive_variants},
  **{key: FailedYieldRightofWay_OperatedInattentive for key in FailedYieldRightofWay_OperatedInattentive_variants},
  **{key: FailedYieldRightofWay_ImproperPassing for key in FailedYieldRightofWay_ImproperPassing_variants},
  **{key: FailedYieldRightofWay_OperatedRecklessAggressive for key in FailedYieldRightofWay_OperatedRecklessAggressive_variants},
  **{key: FailedYieldRightofWay_FollowedTooClosely for key in FailedYieldRightofWay_FollowedTooClosely_variants},
  **{key: FailedYieldRightofWay_RanStopSign for key in FailedYieldRightofWay_RanStopSign_variants},
  **{key: FailedYieldRightofWay_ImproperTurn for key in FailedYieldRightofWay_ImproperTurn_variants},
  **{key: FailedYieldRightofWay_ImproperBacking for key in FailedYieldRightofWay_ImproperBacking_variants},
  **{key: FailedYieldRightofWay_RanRedLight for key in FailedYieldRightofWay_RanRedLight_variants},
  **{key: Failed_YieldRightofWay for key in Failed_YieldRightofWay_variants },
  **{key: OperatedRecklessAggressiver_RanOffRoadway for key in OperatedRecklessAggressiver_RanOffRoadway_variants},
  **{key: ImproperTurn_OperatedInattentiveCarelessNegligent for key in ImproperTurn_OperatedInattentiveCarelessNegligent_variants},
#
  **{key: ImproperTurn_OverCorrectingSteering for key in ImproperTurn_OverCorrectingSteering_variants},
  **{key: ImproperBacking_OperatedInattentiveCareless for key in ImproperBacking_OperatedInattentiveCareless_variants},
  **{key: ImproperPassing_OperatedInattentiveCarelessNegligentErratic for key in ImproperPassing_OperatedInattentiveCarelessNegligentErratic_variants},
  **{key: ImproperPassing_OperatedRecklessAggressive for key in ImproperPassing_OperatedRecklessAggressive_variants},
  **{key: TooFast_ForConditions for key in TooFast_ForConditions_variants},
  **{key: Swerved_DueWind_SlipperySurface for key in Swerved_DueWind_SlipperySurface_variants},
	}


main_merge['Circumstance'] = main_merge['Circumstance'].replace(replacements_circum_individual)
main_merge['Circumstance'] = main_merge['Circumstance'].replace(replacements_circumstance)

In [ ]:
#Crash Replacement for Standardization - Driver Distracted By
#replacements



#Replacing
replacements_Distracted = {
  'NOT DISTRACTED':'Not Distracted',

  'BY OTHER OCCUPANTS': 'By Other Occupants',
  'OTHER DISTRACTION': 'Other Distraction',
  'LOOKED BUT DID NOT SEE':'Looked but Did Not See',
  'UNKNOWN': 'Unknown',
  'NO DRIVER PRESENT':'Driver Not Present',
  'INATTENTIVE OR LOST IN THOUGHT':'Inattentive or Lost in Thought',
  'USING DEVICE OBJECT BROUGHT INTO VEHICLE':'External Device Brought into Vehicle',
  'BY MOVING OBJECT IN VEHICLE':'Moving Object in Vehicle',
  'OTHER ELECTRONIC DEVICE (NAVIGATIONAL PALM PILOT)':'Portable Navigation Device',

  'DISTRACTED BY OUTSIDE PERSON OBJECT OR EVENT':'External Person/Objective/Event',
  'USING OTHER DEVICE CONTROLS INTEGRAL TO VEHICLE':'Using Intergral Device Controls',
  'ADJUSTING AUDIO AND OR CLIMATE CONTROLS':'Adjusting Audio/Climate Controlls',
  'EATING OR DRINKING':'Eating/Drinking',
  #'Other Action (looking away from task, etc.)' :'',
  'OTHER CELLULAR PHONE RELATED':'Other Cellular Phone Related',
  'TALKING OR LISTENING TO CELLULAR PHONE':'Talking/Listening to Cellular Phone',
  #'Talking/listening':'',
  'DIALING CELLULAR PHONE':'Dialing Cellular Phone',
  'TEXTING FROM A CELLULAR PHONE':'Texting',
  'SMOKING RELATED':'Smoking Related',
  }

main_merge['Driver Distracted By'] = main_merge['Driver Distracted By'].replace(replacements_Distracted)

In [ ]:
#Crash Replacement for Standardization - Vehicle Damage Extent
#Replacing
replacements_vdamage = { 'FUNCTIONAL':'Functional',
'DISABLING':'Disabling',
'SUPERFICIAL':'Superficial',
'NO DAMAGE':'No Damage',
'UNKNOWN': 'Unknown',
'DESTROYED':'Destroyed',
'OTHER':'Other', }

main_merge['Vehicle Damage Extent'] = main_merge['Vehicle Damage Extent'].replace(replacements_vdamage)

In [ ]:
#Crash Replacement for Standardization - Vehicle Movement

#Replacing
replacements_vmovement = {
  'MOVING CONSTANT SPEED':'Moving Constant Speed',
  'MAKING LEFT TURN': 'Turning Left',
  'STOPPED IN TRAFFIC LANE': 'Stopped in Traffic',
  'SLOWING OR STOPPING':'Slowing or Stopping',
  'CHANGING LANES' : 'Changing Lanes',
  'MAKING RIGHT TURN': 'Turning Right',
  'BACKING':'Backing',
  'STARTING FROM LANE':'Starting From Lane',
  'RIGHT TURN ON RED' : 'Right Turn from Red',
  'PASSING':'Overtaking/Passing',
  'ACCELERATING':'Accelerating',
  'STARTING FROM PARKED' : 'Starting from Parked',
  'ENTERING TRAFFIC LANE' : 'Entering Traffic Lane',
  'PARKED' :'Parked',
  'MAKING U TURN' :'Making U-Turn',
  'UNKNOWN': 'Unknown',
  'PARKING': 'Parking',
  'NEGOTIATING A CURVE':'Negotiating a Curve',
  'OTHER': 'Other',
  'SKIDDING': 'Skidding',
  'LEAVING TRAFFIC LANE':'Leaving Traffic Lane',
  'DRIVERLESS MOVING VEH.': 'Driverless Moving Vehicle',
  }

main_merge['Vehicle Movement'] = main_merge['Vehicle Movement'].replace(replacements_vmovement)

In [ ]:
#Crash Replacement for Standardization - Road Grade

#Replacing
replacements_roadgrade = {
  'HILL CREST':'Hillcrest',
  'UNKNOWN':'Unknown',
  'LEVEL': 'Level',
  'GRADE DOWNHILL':'Downhill',
  'HILL UPHILL': 'Uphill',
  'DIP SAG':'Sag(bottom)',
  'OTHER':'Other',
  'ON BRIDGE': 'On a Bridge',
 }

main_merge['Road Grade'] = main_merge['Road Grade'].replace(replacements_roadgrade)

In [ ]:
#Crash Replacement for Standardization - Vehicle Body Type

#Replacing
replacements_bodytype = {
  'PASSENGER CAR': 'Passenger Car',
  'SCHOOL BUS': 'Bus - School',
  'TRANSIT BUS': 'Bus - Transit',
  '(SPORT) UTILITY VEHICLE': 'Sport Utility Vehicle',
  'STATION WAGON': 'Station Wagon',
  'PICKUP TRUCK': 'Pickup',
  'AUTOCYCLE':'Autocycle',
  'TRUCK TRACTOR':'Truck Tractor',
  'MOPED':'Moped Or motorized bicycle',
  'SNOWMOBILE':'Snowmobile',
  'CROSS COUNTRY BUS' :'Bus - Cross Country',
  'FARM VEHICLE' :'Farm Equipment (Tractor, combine harvester, etc.)',
  'CARGO VAN/LIGHT TRUCK 2 AXLES (OVER 10,000LBS (4,536 KG))' :'Van - Cargo',
  'ALL TERRAIN VEHICLE (ATV)' : 'All-Terrain Vehicle/All-Terrain Cycle (ATV/ATC)',
  'LOW SPEED VEHICLE':'Low Speed Vehicle',

  'OTHER BUS': 'Bus - Other Type',

  'UNKNOWN': 'Unknown',
  'OTHER': 'Other',
  'VAN':'Van',
  'Van - Passenger (&lt;9 Seats)': 'Van - Passenger (<9 Seats)',}

main_merge['Vehicle Body Type'] = main_merge['Vehicle Body Type'].replace(replacements_bodytype)

In [ ]:
#Crash Replacement for Standardization - First Harmful Event
#replacements



#Replacing
replacements_harmfulevent = {
  'UNKNOWN': 'Unknown',
  'PARKED VEHICLE':'Parked Vehicle',
  'PEDESTRIAN':'Pedestrian',
  'FELL JUMPED FROM MOTOR VEHICLE':'Fell Jumped from Motor Vehicle',
  'EXPLOSION OR FIRE':'Explosion Or Fire',
  'OTHER PEDALCYCLE': 'Pedalcycle',
  'OTHER NON COLLISION':'Other Non-Collision',
  'JACKKNIFE':'Jackknife',
  'SPILLED CARGO':'Spilled Cargo',
  'OVERTURN':'Overturn/Rollover',
  'UNITS SEPARATED':'Units Separated',
  'RAILWAY TRAIN':'Railway Vehicle (train, engine)',
  'THROWN OR FALLING OBJECT':'Thrown Or Falling Object',
  'FIXED OBJECT':'Other Fixed Object (wall, building, tunnel, etc.)',
  'ANIMAL':'Animal (live)',
  'OTHER':'Other',
  }

main_merge['First Harmful Event'] = main_merge['First Harmful Event'].replace(replacements_harmfulevent)

In [ ]:
#Crash Replacement for Standardization - Second Harmful Event

main_merge['Second Harmful Event'] = main_merge['Second Harmful Event'].replace(replacements_harmfulevent )

In [ ]:
#Crash Replacement for Standardization - Junction

#Replacing
replacements_junction_ind = {
  'CROSSOVER RELATED':'Crossover-Related',
  'OTHER': 'Other',
  'UNKNOWN': 'Unknown',
  'RAILWAY GRADE CROSSING': 'Railway Grade Crossing',
  'NON INTERSECTION':  'Non-Junction',
  }

intersection = 'Intersection'
intersection_junction_variants ={
  'INTERSECTION RELATED',
  'INTERSECTION',
  'Intersection or Related',
  }


replacements_junction = {
  **{key: intersection for key in intersection_junction_variants},}

main_merge['Junction'] = main_merge['Junction'].replace(replacements_junction_ind)
main_merge['Junction'] = main_merge['Junction'].replace(replacements_junction)

In [ ]:
#Crash Replacement for Standardization - Intersection Type

#Replacing
replacements_inttype_ind = {
  'UNKNOWN': 'Unknown',
  'OTHER': 'Other',
  }

roundabout = 'Roundabout/Traffic Circle'
roundabout_variants ={
 'Roundabout/Traffic Circle',
 'TRAFFIC CIRCLE',
 'ROUNDABOUT',
  }


replacements_inttype = {
  **{key: roundabout for key in roundabout_variants},}

main_merge['Intersection Type'] = main_merge['Intersection Type'].replace(replacements_inttype_ind)
main_merge['Intersection Type'] = main_merge['Intersection Type'].replace(replacements_inttype)

In [ ]:
#Crash Replacement for Standardization - Road Alignment

#Replacing
replacements_roadalignment = {
  'STRAIGHT': 'Straight',
  'UNKNOWN':'Unknown',
  'OTHER': 'Other',
  'CURVE LEFT':'Curve Left',
  'CURVE RIGHT':'Curve Right',
  }

main_merge['Road Alignment'] = main_merge['Road Alignment'].replace(replacements_roadalignment)

In [ ]:
#Crash Replacement for Standardization - Road Condition

#Replacing
replacements_roadCondition = {
  'NO DEFECTS':'No Defects',
  'VIEW OBSTRUCTED':'View Obstructed',
  'OBSTRUCTION NOT LIGHTED':'Obstruction Not Lighted',
  'OBSTRUCTION NOT SIGNALED':'Obstruction Not Signaled',
  'HOLES RUTS ETC':'Holes Ruts Etc',
  'LOOSE SURFACE MATERIAL':'Loose Surface Material',
  'FOREIGN MATERIAL':'Foreign Material',
  'SHOULDER DEFECT':'Shoulder Defect',

  'UNKNOWN':'Unknown',
  'OTHER':'Other', }

main_merge['Road Condition'] = main_merge['Road Condition'].replace(replacements_roadCondition)

In [ ]:
#Crash Replacement for Standardization -Lane Direction

replacements_lanedir = {
  'Unknown, Westbound':  'Westbound',
  'Not On Roadway, Southbound, Unknown': 'Not On Roadway, Southbound',
  'Not On Roadway, Unknown': 'Not On Roadway',
  'Northbound, Unknown': 'Northbound',
  'Eastbound, Unknown':'Eastbound',
  'Southbound, Unknown': 'Southbound',
  'Northbound, Not On Roadway, Unknown': 'Northbound, Not On Roadway',
  }


main_merge['Lane Direction'] = main_merge['Lane Direction'].replace(replacements_lanedir)

In [ ]:
#Crash Replacement for Standardization - Lane Type

#Replacing
replacements_lanetype_ind = {
  'UNKNOWN':'Unknown',
  'CROSSOVER AREA':'Crossover Area',
  'RIGHT TURN LANE':'Right Turn Lane',
  'LEFT TURN LANE':'Left Turn Lane',
  'MEDIAN AREA':'Median Area',
  'OFF ROAD':'Off Road',
  'OTHER': 'Other',
  'OUTSIDE RIGHT OF WAY':'Outside Right Of Way',

  'GORE AREA':'Gore Area',
  'SEPARATOR':'Separator',
  'ACCELERATION LANE':'Acceleration Lane',
  'DECELERATION LANE': 'Deceleration',
  'Not Applicable, Other':'Not Applicable',
  'Lane 1, Other':'Lane 1',
  'Lane 2, Other':'Lane 2',
  'Lane 3, Other':'Lane 3',

  }

lanetype_shoulder = 'Shoulder Area'
lanetype_shoulder_variants ={
  'Shoulder Area',
  'Shoulder Area, Unknown',
  'Other, Shoulder Area',
  'SHOULDER AREA',
  }

lanetype_ramp = 'On Ramp'
lanetype_ramp_variants ={
  'ON RAMP',
  'On Ramp',
  'On Ramp, Other',
  'On Ramp, Unknown',
  }

replacements_lanetype = {
  **{key: lanetype_shoulder for key in lanetype_shoulder_variants},
  **{key: lanetype_ramp for key in lanetype_ramp_variants},
  }

main_merge['Lane Type'] = main_merge['Lane Type'].replace(replacements_lanetype_ind)
main_merge['Lane Type'] = main_merge['Lane Type'].replace(replacements_lanetype)

In [ ]:
# Looking int chosen datatset
look_in = main_merge

#look_in.info()
print('~~~~~~~~~~~~~~~~~~~~~')



#Checking what is null and uniques vaules in column
#Look into which colume
col ='Injury Severity'

#Nulls
print('Here are the number of Null in each feature\n',look_in.isnull().sum())
print('~~~~~~~\n')
print('Here are the number of Null in this feature\n',col,look_in[col].isnull().sum())

#unique values
print(f'\n Number of Unique values in {col}: ',(look_in[col].nunique()), '\nThis Colume has:', set(look_in[col]))


~~~~~~~~~~~~~~~~~~~~~
Here are the number of Null in each feature
 Report Number        0
Local Case Number    0
Agency Name          0
ACRS Report Type     0
Crash Date/Time      0
                    ..
Time                 0
Holiday              0
Day Type             0
Day of Week          0
Time of Day          0
Length: 63, dtype: int64
~~~~~~~

Here are the number of Null in this feature
 Injury Severity 27415

 Number of Unique values in Injury Severity:  5 
This Colume has: {'Fatal Injury', nan, 'Suspected Serious Injury', 'Suspected Minor Injury', 'Possible Injury', 'No Apparent Injury'}


In [ ]:
# Check all features and unique values
unique__whole = {}

exclude_columns = ['Report Number', 'Local Case Number','Crash Date/Time', 'Time',
                   'Latitude','Longitude','Geolocation', 'Distance',
                   'Person ID', 'Vehicle ID', 'Off-Road Description',
                   'Vehicle Model','Vehicle Make','Vehicle Year', 'Vehicle First Impact Location',
                   'Road Name','Cross-Street Name']
#'Lane Direction',
# loop through the columns
for col in main_merge.columns:
  if col not in exclude_columns:
    unique__whole[col] = {
      'unique_count': main_merge[col].nunique(),
      'unique_values': main_merge[col].unique().tolist()
  }

# Convert to dataframe
unique__whole_df = pd.DataFrame.from_dict(unique__whole, orient='index')

In [ ]:
# Save it as a csv
unique__whole_df.to_csv('unique__whole_df.csv')
print("Dictionary saved to unique_values_info.csv successfully!")
print('Features', len(main_merge.columns), '\nSamples',len(main_merge))

Dictionary saved to unique_values_info.csv successfully!
Features 62 
Samples 217809


In [ ]:
#Save new CSV
main_merge.to_csv('Merged_Collison_Data.csv',)